In [1]:
import subprocess
import sys
import urllib.request
import gzip
from Bio import SeqIO
import os
import glob
import numpy as np
from os import listdir
import pandas as pd
import jellyfish
import shlex
import mmh3

In [63]:
cd -

/hobbitholes/waymoot/hchen/acheron/acheron


In [64]:
all_files = listdir("data")
all_file_directories = []
for file in all_files:
    yes = "data/"  + str(file)
    all_file_directories.append(yes)

In [65]:
all_file_directories[1]

'data/GCF_005938105.1_ASM593810v1.genomic.fna.gz'

In [66]:
all_files_data = listdir
all_file_names = []
for file in all_files:
    all_file_names.append(file)
zcat_all_files = ''
for file_name in all_file_names:
    zcat_all_files += '<(zcat ' + file_name + ') '
print(zcat_all_files)

<(zcat GCF_900446045.1_58932_D01.genomic.fna.gz) <(zcat GCF_005938105.1_ASM593810v1.genomic.fna.gz) <(zcat GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.genomic.fna.gz) <(zcat GCF_022024335.1_ASM2202433v1.genomic.fna.gz) <(zcat GCF_002191835.1_ASM219183v1.genomic.fna.gz) <(zcat GCF_900491905.1_96-9626.genomic.fna.gz) <(zcat GCF_001702305.1_ASM170230v1.genomic.fna.gz) <(zcat GCF_947242725.1_B133.genomic.fna.gz) <(zcat GCF_009733255.1_Brucella_abortus_381.genomic.fna.gz) <(zcat GCF_001402105.1_ASM140210v1.genomic.fna.gz) <(zcat GCF_008932425.1_ASM893242v1.genomic.fna.gz) <(zcat GCF_000366445.1_Bruc_abor_88_226_V1.genomic.fna.gz) <(zcat GCF_900092405.1_10RB9215v1.genomic.fna.gz) <(zcat GCF_006507455.1_ASM650745v1.genomic.fna.gz) <(zcat GCF_000740295.1_ASM74029v1.genomic.fna.gz) <(zcat GCF_003937445.1_ASM393744v1.genomic.fna.gz) <(zcat GCF_000414015.1_Bruc_abor_biovar_1_82-2330_V1.genomic.fna.gz) <(zcat GCF_000370025.1_Bruc_abor_63_144_V1.genomic.fna.gz) <(zcat GCF_000413655.1_Bruc_abor_b

In [ ]:
jellyfish count -m 25 -s 100M -t 10 -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz) <(zcat GCF_005938105.1_ASM593810v1.genomic.fna.gz) <(zcat GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.genomic.fna.gz) <(zcat GCF_022024335.1_ASM2202433v1.genomic.fna.gz) <(zcat GCF_002191835.1_ASM219183v1.genomic.fna.gz) <(zcat GCF_900491905.1_96-9626.genomic.fna.gz) <(zcat GCF_001702305.1_ASM170230v1.genomic.fna.gz) <(zcat GCF_947242725.1_B133.genomic.fna.gz) <(zcat GCF_009733255.1_Brucella_abortus_381.genomic.fna.gz) <(zcat GCF_001402105.1_ASM140210v1.genomic.fna.gz) <(zcat GCF_008932425.1_ASM893242v1.genomic.fna.gz) <(zcat GCF_000366445.1_Bruc_abor_88_226_V1.genomic.fna.gz) <(zcat GCF_900092405.1_10RB9215v1.genomic.fna.gz) <(zcat GCF_006507455.1_ASM650745v1.genomic.fna.gz) <(zcat GCF_000740295.1_ASM74029v1.genomic.fna.gz) <(zcat GCF_003937445.1_ASM393744v1.genomic.fna.gz) <(zcat GCF_000414015.1_Bruc_abor_biovar_1_82-2330_V1.genomic.fna.gz) <(zcat GCF_000370025.1_Bruc_abor_63_144_V1.genomic.fna.gz) <(zcat GCF_000413655.1_Bruc_abor_biovar_1_90-0742_V1.genomic.fna.gz) <(zcat GCF_947242185.1_B70.genomic.fna.gz) <(zcat GCF_026014005.1_ASM2601400v1.genomic.fna.gz) <(zcat GCF_947242325.1_B10.genomic.fna.gz) <(zcat GCF_008932705.1_ASM893270v1.genomic.fna.gz) <(zcat GCF_000399725.1_Bruc_abor_I103_UK3_01_V1.genomic.fna.gz) <(zcat GCF_000740175.1_ASM74017v1.genomic.fna.gz) <(zcat GCF_904066055.1_WS-Be-9.genomic.fna.gz) <(zcat GCF_006506215.1_ASM650621v1.genomic.fna.gz) <(zcat GCF_000367305.1_Bruc_cani_F7_05A_V1.genomic.fna.gz) <(zcat GCF_006507585.1_ASM650758v1.genomic.fna.gz) <(zcat GCF_000371225.1_Bruc_suis_F7_06-1_V1.genomic.fna.gz) <(zcat GCF_947242455.1_B18.genomic.fna.gz) <(zcat GCF_900454225.1_34576_H01.genomic.fna.gz) <(zcat GCF_947242495.1_B112.genomic.fna.gz) <(zcat GCF_001937895.1_ASM193789v1.genomic.fna.gz) <(zcat GCF_900446085.1_59302_C02.genomic.fna.gz) <(zcat GCF_000740415.1_ASM74041v1.genomic.fna.gz) <(zcat GCF_947242245.1_B143.genomic.fna.gz) <(zcat GCF_001856365.1_ASM185636v1.genomic.fna.gz) <(zcat GCF_006507245.1_ASM650724v1.genomic.fna.gz) <(zcat GCF_947242395.1_B120.genomic.fna.gz) <(zcat GCF_017177515.1_ASM1717751v1.genomic.fna.gz) <(zcat GCF_000365905.1_Bruc_ovis_IntaBari-2010-47-871_V1.genomic.fna.gz) <(zcat GCF_900236535.1_Bru_genome4.genomic.fna.gz) <(zcat GCF_002245235.1_ASM224523v1.genomic.fna.gz) <(zcat GCF_000413695.1_Bruc_abor_biovar_2_90-0737_V1.genomic.fna.gz) <(zcat GCF_900446075.1_59071_A01.genomic.fna.gz) <(zcat GCF_000685375.1_BMIND1_v1.genomic.fna.gz) <(zcat GCF_947241785.1_B146.genomic.fna.gz) <(zcat GCF_017177475.1_ASM1717747v1.genomic.fna.gz) <(zcat GCF_947241885.1_B140.genomic.fna.gz) <(zcat GCF_947242625.1_B128.genomic.fna.gz) <(zcat GCF_024720755.1_ASM2472075v1.genomic.fna.gz) <(zcat GCF_900236495.1_Bru_genome46.genomic.fna.gz) <(zcat GCF_023651275.1_ASM2365127v1.genomic.fna.gz) <(zcat GCF_001702335.1_ASM170233v1.genomic.fna.gz) <(zcat GCF_000236255.1_ASM23625v1.genomic.fna.gz) <(zcat GCF_947242425.1_B114.genomic.fna.gz) <(zcat GCF_002943495.1_ASM294349v1.genomic.fna.gz) <(zcat GCF_000366905.1_Bruc_meli_CNGB_290_V1.genomic.fna.gz) <(zcat GCF_000473805.1_Brucella82.genomic.fna.gz) <(zcat GCF_000413815.1_Bruc_abor_biovar_1_B10-0018_V1.genomic.fna.gz) <(zcat GCF_000740395.1_ASM74039v1.genomic.fna.gz) <(zcat GCF_000365985.1_Bruc_ovis_IntaBari-2008-114-542_V1.genomic.fna.gz) <(zcat GCF_000371045.1_Bruc_sp_UK1_97_V1.genomic.fna.gz) <(zcat GCF_002245225.1_ASM224522v1.genomic.fna.gz) <(zcat GCF_000371305.1_Bruc_suis_F8_06-3_V1.genomic.fna.gz) <(zcat GCF_000158995.1_ASM15899v1.genomic.fna.gz) <(zcat GCF_026413105.1_ASM2641310v1.genomic.fna.gz) <(zcat GCF_001511795.1_BVF965.genomic.fna.gz) <(zcat GCF_006376675.1_ASM637667v1.genomic.fna.gz) <(zcat GCF_947241895.1_B50.genomic.fna.gz) <(zcat GCF_000370065.1_Bruc_abor_63_294_V1.genomic.fna.gz) <(zcat GCF_000221005.1_ASM22100v1.genomic.fna.gz) <(zcat GCF_006506485.1_ASM650648v1.genomic.fna.gz) <(zcat GCF_006507765.1_ASM650776v1.genomic.fna.gz) <(zcat GCF_006506225.1_ASM650622v1.genomic.fna.gz) <(zcat GCF_900236395.1_Bru_genome41.genomic.fna.gz) <(zcat GCF_947242615.1_B85.genomic.fna.gz) <(zcat GCF_000370665.1_Bruc_meli_F10_06-16_V1.genomic.fna.gz) <(zcat GCF_003194855.1_ASM319485v1.genomic.fna.gz) <(zcat GCF_000366725.1_Bruc_abor_NI622_V1.genomic.fna.gz) <(zcat GCF_000740435.1_ASM74043v1.genomic.fna.gz) <(zcat GCF_006506295.1_ASM650629v1.genomic.fna.gz) <(zcat GCF_023796775.1_ASM2379677v1.genomic.fna.gz) <(zcat GCF_006376685.1_ASM637668v1.genomic.fna.gz) <(zcat GCF_000472165.1_229E.genomic.fna.gz) <(zcat GCF_023651575.1_ASM2365157v1.genomic.fna.gz) <(zcat GCF_002191395.1_ASM219139v1.genomic.fna.gz) <(zcat GCF_001038665.1_ASM103866v1.genomic.fna.gz) <(zcat GCF_004684165.1_ASM468416v1.genomic.fna.gz) <(zcat GCF_023651015.1_ASM2365101v1.genomic.fna.gz) <(zcat GCF_000366965.1_Bruc_meli_F3_02_V1.genomic.fna.gz) <(zcat GCF_000367005.1_Bruc_meli_F6_05-6_V1.genomic.fna.gz) <(zcat GCF_017177325.1_ASM1717732v1.genomic.fna.gz) <(zcat GCF_021083425.1_ASM2108342v1.genomic.fna.gz) <(zcat GCF_900446065.1_58932_F01.genomic.fna.gz) <(zcat GCF_006517325.1_ASM651732v1.genomic.fna.gz) <(zcat GCF_000371245.1_Bruc_suis_F7_06-2_V1.genomic.fna.gz) <(zcat GCF_000157795.1_ASM15779v1.genomic.fna.gz) <(zcat GCF_003194795.1_ASM319479v1.genomic.fna.gz) <(zcat GCF_003989875.1_ASM398987v1.genomic.fna.gz) <(zcat GCF_028129075.1_ASM2812907v1.genomic.fna.gz) <(zcat GCF_002245205.1_ASM224520v1.genomic.fna.gz) <(zcat GCF_900446095.1_59071_B01.genomic.fna.gz) <(zcat GCF_947242205.1_B144.genomic.fna.gz) <(zcat GCF_008761575.1_ASM876157v1.genomic.fna.gz) <(zcat GCF_000742005.1_ASM74200v1.genomic.fna.gz) <(zcat GCF_017177475.2_Bm-511.genomic.fna.gz) <(zcat GCF_947242525.1_B57.genomic.fna.gz) <(zcat GCF_947242025.1_B130.genomic.fna.gz) <(zcat GCF_006507565.1_ASM650756v1.genomic.fna.gz) <(zcat GCF_001996385.1_ASM199638v1.genomic.fna.gz) <(zcat GCF_904066065.1_WS-Be-70.genomic.fna.gz) <(zcat GCF_009361155.1_ASM936115v1.genomic.fna.gz) <(zcat GCF_022426845.1_ASM2242684v1.genomic.fna.gz) <(zcat GCF_017177425.1_ASM1717742v1.genomic.fna.gz) <(zcat GCF_006506465.1_ASM650646v1.genomic.fna.gz) <(zcat GCF_000370645.1_Bruc_meli_F1_06_B10_V1.genomic.fna.gz) <(zcat GCF_006506725.1_ASM650672v1.genomic.fna.gz) <(zcat GCF_023651415.1_ASM2365141v1.genomic.fna.gz) <(zcat GCF_000480275.1_Bruc_cani_96-7258_V1.genomic.fna.gz) <(zcat GCF_023651555.1_ASM2365155v1.genomic.fna.gz) <(zcat GCF_900095155.1_BR141012304v1.genomic.fna.gz) <(zcat GCF_024127925.1_ASM2412792v1.genomic.fna.gz) <(zcat GCF_000366405.1_Bruc_abor_863_67_V1.genomic.fna.gz) <(zcat GCF_000370205.1_Bruc_abor_78_32_V1.genomic.fna.gz) <(zcat GCF_900236355.1_Bru_genome20.genomic.fna.gz) <(zcat GCF_003994155.1_ASM399415v1.genomic.fna.gz) <(zcat GCF_009601725.1_ASM960172v1.genomic.fna.gz) <(zcat GCF_947242335.1_B115.genomic.fna.gz) <(zcat GCF_016484075.1_ASM1648407v1.genomic.fna.gz) <(zcat GCF_000370385.1_Bruc_abor_F10_05-11_V1.genomic.fna.gz) <(zcat GCF_947242795.1_B137.genomic.fna.gz) <(zcat GCF_000157735.1_ASM15773v1.genomic.fna.gz) <(zcat GCF_000366945.1_Bruc_meli_F2_06-6_V1.genomic.fna.gz) <(zcat GCF_000366385.1_Bruc_abor_85_140_V1.genomic.fna.gz) <(zcat GCF_000370465.1_Bruc_abor_F6_05-4_V1.genomic.fna.gz) <(zcat GCF_000591005.1_548.genomic.fna.gz) <(zcat GCF_947242375.1_B9.genomic.fna.gz) <(zcat GCF_000878465.1_ASM87846v1.genomic.fna.gz) <(zcat GCF_947242155.1_B76.genomic.fna.gz) <(zcat GCF_002192115.1_ASM219211v1.genomic.fna.gz) <(zcat GCF_006507965.1_ASM650796v1.genomic.fna.gz) <(zcat GCF_900446015.1_58932_C01.genomic.fna.gz) <(zcat GCF_002895125.1_ASM289512v1.genomic.fna.gz) <(zcat GCF_001856215.1_ASM185621v1.genomic.fna.gz) <(zcat GCF_947242415.1_B36.genomic.fna.gz) <(zcat GCF_006506575.1_ASM650657v1.genomic.fna.gz) <(zcat GCF_001715445.1_ASM171544v1.genomic.fna.gz) <(zcat GCF_001938165.1_ASM193816v1.genomic.fna.gz) <(zcat GCF_000292005.1_ASM29200v2.genomic.fna.gz) <(zcat GCF_000413575.1_Bruc_abor_biovar_1_84-0928_V1.genomic.fna.gz) <(zcat GCF_002191595.1_ASM219159v1.genomic.fna.gz) <(zcat GCF_009823705.1_ASM982370v1.genomic.fna.gz) <(zcat GCF_001876955.1_ASM187695v1.genomic.fna.gz) <(zcat GCF_001856345.1_ASM185634v1.genomic.fna.gz) <(zcat GCF_016126135.1_ASM1612613v1.genomic.fna.gz) <(zcat GCF_014884505.1_ASM1488450v1.genomic.fna.gz) <(zcat GCF_003205535.1_ASM320553v1.genomic.fna.gz) <(zcat GCF_947242705.1_B47.genomic.fna.gz) <(zcat GCF_947242405.1_B119.genomic.fna.gz) <(zcat GCF_000292105.1_ASM29210v2.genomic.fna.gz) <(zcat GCF_947242535.1_B11.genomic.fna.gz) <(zcat GCF_008932675.1_ASM893267v1.genomic.fna.gz) <(zcat GCF_006506395.1_ASM650639v1.genomic.fna.gz) <(zcat GCF_000245855.1_Bruc_abor_bv1_NI486_V1.genomic.fna.gz) <(zcat GCF_002214285.1_ASM221428v1.genomic.fna.gz) <(zcat GCF_947242545.1_B106.genomic.fna.gz) <(zcat GCF_003989975.1_ASM398997v1.genomic.fna.gz) <(zcat GCF_000413995.1_Bruc_abor_biovar_1_87-2211_V1.genomic.fna.gz) <(zcat GCF_009797905.1_ASM979790v1.genomic.fna.gz) <(zcat GCF_002278585.1_ASM227858v1.genomic.fna.gz) <(zcat GCF_001402025.1_ASM140202v1.genomic.fna.gz) <(zcat GCF_947242595.1_B8.genomic.fna.gz) <(zcat GCF_009664915.1_ASM966491v1.genomic.fna.gz) <(zcat GCF_006506995.1_ASM650699v1.genomic.fna.gz) <(zcat GCF_002266785.1_ASM226678v1.genomic.fna.gz) <(zcat GCF_008761595.1_ASM876159v1.genomic.fna.gz) <(zcat GCF_000192725.1_ASM19272v1.genomic.fna.gz) <(zcat GCF_023651055.1_ASM2365105v1.genomic.fna.gz) <(zcat GCF_000245875.1_Bruc_abor_bv1_NI474_V1.genomic.fna.gz) <(zcat GCF_019998625.1_ASM1999862v1.genomic.fna.gz) <(zcat GCF_003989895.1_ASM398989v1.genomic.fna.gz) <(zcat GCF_006506985.1_ASM650698v1.genomic.fna.gz) <(zcat GCF_002591255.1_ASM259125v1.genomic.fna.gz) <(zcat GCF_947241955.1_B141.genomic.fna.gz) <(zcat GCF_003675365.1_ASM367536v1.genomic.fna.gz) <(zcat GCF_000413795.1_Bruc_abor_biovar_1_01-0065_V1.genomic.fna.gz) <(zcat GCF_947242645.1_B52.genomic.fna.gz) <(zcat GCF_006507975.1_ASM650797v1.genomic.fna.gz) <(zcat GCF_008932505.1_ASM893250v1.genomic.fna.gz) <(zcat GCF_000366085.1_Bruc_suis_63_252_V1.genomic.fna.gz) <(zcat GCF_000371065.1_Bruc_sp_UK40_99_V1.genomic.fna.gz) <(zcat GCF_026256585.1_ASM2625658v1.genomic.fna.gz) <(zcat GCF_006507545.1_ASM650754v1.genomic.fna.gz) <(zcat GCF_001937885.1_ASM193788v1.genomic.fna.gz) <(zcat GCF_006508025.1_ASM650802v1.genomic.fna.gz) <(zcat GCF_000559065.1_SJY1.genomic.fna.gz) <(zcat GCF_000742275.1_ASM74227v1.genomic.fna.gz) <(zcat GCF_000160275.1_ASM16027v1.genomic.fna.gz) <(zcat GCF_000370105.1_Bruc_abor_64_81_V1.genomic.fna.gz) <(zcat GCF_000367045.1_Bruc_meli_UK19_04_V1.genomic.fna.gz) <(zcat GCF_002204295.1_ASM220429v1.genomic.fna.gz) <(zcat GCF_009664925.1_ASM966492v1.genomic.fna.gz) <(zcat GCF_016126205.1_ASM1612620v1.genomic.fna.gz) <(zcat GCF_000371185.1_Bruc_suis_5_05-10_V1.genomic.fna.gz) <(zcat GCF_002290725.1_ASM229072v1.genomic.fna.gz) <(zcat GCF_006507235.1_ASM650723v1.genomic.fna.gz) <(zcat GCF_000177135.1_ASM17713v1.genomic.fna.gz) <(zcat GCF_002191775.1_ASM219177v1.genomic.fna.gz) <(zcat GCF_006507015.1_ASM650701v1.genomic.fna.gz) <(zcat GCF_002192155.1_ASM219215v1.genomic.fna.gz) <(zcat GCF_900417255.1_00-3099-5733.genomic.fna.gz) <(zcat GCF_902825325.1_OCH-ISR2.genomic.fna.gz) <(zcat GCF_015832115.1_ASM1583211v1.genomic.fna.gz) <(zcat GCF_001307425.1_ASM130742v1.genomic.fna.gz) <(zcat GCF_001302495.1_ASM130249v1.genomic.fna.gz) <(zcat GCF_025118245.1_ASM2511824v1.genomic.fna.gz) <(zcat GCF_000157695.1_ASM15769v1.genomic.fna.gz) <(zcat GCF_002191495.1_ASM219149v1.genomic.fna.gz) <(zcat GCF_947242685.1_B7.genomic.fna.gz) <(zcat GCF_017177825.1_ASM1717782v1.genomic.fna.gz) <(zcat GCF_000742255.1_ASM74225v1.genomic.fna.gz) <(zcat GCF_008932825.1_ASM893282v1.genomic.fna.gz) <(zcat GCF_006506745.1_ASM650674v1.genomic.fna.gz) <(zcat GCF_001971805.1_ASM197180v1.genomic.fna.gz) <(zcat GCF_000370405.1_Bruc_abor_F10_06-3_V1.genomic.fna.gz) <(zcat GCF_009733435.1_Brucella_abortus_I-12.genomic.fna.gz) <(zcat GCF_003994115.1_ASM399411v1.genomic.fna.gz) <(zcat GCF_003993965.1_ASM399396v1.genomic.fna.gz) <(zcat GCF_000698285.1_ASM69828v1.genomic.fna.gz) <(zcat GCF_000245975.1_Bruc_abor_bv1_NI259_V1.genomic.fna.gz) <(zcat GCF_000370325.1_Bruc_abor_877_67_V1.genomic.fna.gz) <(zcat GCF_017355465.1_ASM1735546v1.genomic.fna.gz) <(zcat GCF_000238195.1_ASM23819v1.genomic.fna.gz) <(zcat GCF_947242295.1_B67.genomic.fna.gz) <(zcat GCF_000366245.1_Bruc_suis_94_11_V1.genomic.fna.gz) <(zcat GCF_000366465.1_Bruc_abor_90_50_V1.genomic.fna.gz) <(zcat GCF_014640615.1_ASM1464061v1.genomic.fna.gz) <(zcat GCF_002631235.1_ASM263123v1.genomic.fna.gz) <(zcat GCF_025846885.1_ASM2584688v1.genomic.fna.gz) <(zcat GCF_006507945.1_ASM650794v1.genomic.fna.gz) <(zcat GCF_003194755.1_ASM319475v1.genomic.fna.gz) <(zcat GCF_006507375.1_ASM650737v1.genomic.fna.gz) <(zcat GCF_000370925.1_Bruc_sp_56_94_V1.genomic.fna.gz) <(zcat GCF_000371125.1_Bruc_suis_63_261_V1.genomic.fna.gz) <(zcat GCF_947242865.1_B103.genomic.fna.gz) <(zcat GCF_001401925.1_ASM140192v1.genomic.fna.gz) <(zcat GCF_000366025.1_Bruc_abor_RB51_V1.genomic.fna.gz) <(zcat GCF_947242765.1_B23.genomic.fna.gz) <(zcat GCF_026586705.1_ASM2658670v1.genomic.fna.gz) <(zcat GCF_947242475.1_B25.genomic.fna.gz) <(zcat GCF_006507695.1_ASM650769v1.genomic.fna.gz) <(zcat GCF_001296965.1_ASM129696v1.genomic.fna.gz) <(zcat GCF_947241775.1_B71.genomic.fna.gz) <(zcat GCF_014197065.1_ASM1419706v1.genomic.fna.gz) <(zcat GCF_003989965.1_ASM398996v1.genomic.fna.gz) <(zcat GCF_002191295.1_ASM219129v1.genomic.fna.gz) <(zcat GCF_003993905.1_ASM399390v1.genomic.fna.gz) <(zcat GCF_000369945.1_Bruc_abor_544_V1.genomic.fna.gz) <(zcat GCF_008932385.1_ASM893238v1.genomic.fna.gz) <(zcat GCF_000370545.1_Bruc_abor_NI495a_V1.genomic.fna.gz) <(zcat GCF_002291305.1_ASM229130v1.genomic.fna.gz) <(zcat GCF_023651355.1_ASM2365135v1.genomic.fna.gz) <(zcat GCF_900236335.1_Bru_genome7.genomic.fna.gz) <(zcat GCF_016887525.1_ASM1688752v1.genomic.fna.gz) <(zcat GCF_003516065.1_ASM351606v1.genomic.fna.gz) <(zcat GCF_000712255.1_Bneotomae2.0.genomic.fna.gz) <(zcat GCF_017177535.2_ASM1717753v2.genomic.fna.gz) <(zcat GCF_000370225.1_Bruc_abor_80_101_V1.genomic.fna.gz) <(zcat GCF_000292205.1_V02_for_genome_sequence_of_BCB013.genomic.fna.gz) <(zcat GCF_006506125.1_ASM650612v1.genomic.fna.gz) <(zcat GCF_006507645.1_ASM650764v1.genomic.fna.gz) <(zcat GCF_947242855.1_B156.genomic.fna.gz) <(zcat GCF_000742055.1_ASM74205v1.genomic.fna.gz) <(zcat GCF_002191475.1_ASM219147v1.genomic.fna.gz) <(zcat GCF_002025625.1_ASM202562v1.genomic.fna.gz) <(zcat GCF_947242825.1_B154.genomic.fna.gz) <(zcat GCF_008932305.1_ASM893230v1.genomic.fna.gz) <(zcat GCF_008932435.1_ASM893243v1.genomic.fna.gz) <(zcat GCF_947242035.1_B5.genomic.fna.gz) <(zcat GCF_947242775.1_B19.genomic.fna.gz) <(zcat GCF_001043295.1_ASM104329v1.genomic.fna.gz) <(zcat GCF_002291365.1_ASM229136v1.genomic.fna.gz) <(zcat GCF_900236515.1_Bru_genome43.genomic.fna.gz) <(zcat GCF_017311425.1_ASM1731142v1.genomic.fna.gz) <(zcat GCF_014884565.1_ASM1488456v1.genomic.fna.gz) <(zcat GCF_023650975.1_ASM2365097v1.genomic.fna.gz) <(zcat GCF_947242225.1_B82.genomic.fna.gz) <(zcat GCF_003994105.1_ASM399410v1.genomic.fna.gz) <(zcat GCF_009017375.1_ASM901737v1.genomic.fna.gz) <(zcat GCF_023651475.1_ASM2365147v1.genomic.fna.gz) <(zcat GCF_006506385.1_ASM650638v1.genomic.fna.gz) <(zcat GCF_006507775.1_ASM650777v1.genomic.fna.gz) <(zcat GCF_000367105.1_Bruc_sp_UK5_01_V1.genomic.fna.gz) <(zcat GCF_000413495.1_Bruc_abor_biovar_1_76-1413_V1.genomic.fna.gz) <(zcat GCF_004137275.1_ASM413727v1.genomic.fna.gz) <(zcat GCF_000369965.1_Bruc_abor_600_64_V1.genomic.fna.gz) <(zcat GCF_947242655.1_B117.genomic.fna.gz) <(zcat GCF_000480315.1_Bruc_suis_06-988-1656_V1.genomic.fna.gz) <(zcat GCF_023651195.1_ASM2365119v1.genomic.fna.gz) <(zcat GCF_000365665.1_Bruc_abor_NI240_V1.genomic.fna.gz) <(zcat GCF_008932615.1_ASM893261v1.genomic.fna.gz) <(zcat GCF_006507175.1_ASM650717v1.genomic.fna.gz) <(zcat GCF_003253375.1_ASM325337v1.genomic.fna.gz) <(zcat GCF_000292065.1_ASM29206v1.genomic.fna.gz) <(zcat GCF_000292025.1_ASM29202v2.genomic.fna.gz) <(zcat GCF_006507555.1_ASM650755v1.genomic.fna.gz) <(zcat GCF_947242165.1_B68.genomic.fna.gz) <(zcat GCF_900446005.1_58116_B01.genomic.fna.gz) <(zcat GCF_006507475.1_ASM650747v1.genomic.fna.gz) <(zcat GCF_001856205.1_ASM185620v1.genomic.fna.gz) <(zcat GCF_900491915.1_07-2859-6070.genomic.fna.gz) <(zcat GCF_002192195.1_ASM219219v1.genomic.fna.gz) <(zcat GCF_000742195.1_ASM74219v1.genomic.fna.gz) <(zcat GCF_000940435.1_Brucella_melitensis_MY1483_09.genomic.fna.gz) <(zcat GCF_008932815.1_ASM893281v1.genomic.fna.gz) <(zcat GCF_001402055.1_ASM140205v1.genomic.fna.gz) <(zcat GCF_947242275.1_B37.genomic.fna.gz) <(zcat GCF_000177155.1_ASM17715v1.genomic.fna.gz) <(zcat GCF_002895145.1_ASM289514v1.genomic.fna.gz) <(zcat GCF_000590815.1_ASM59081v1.genomic.fna.gz) <(zcat GCF_001742815.1_ASM174281v1.genomic.fna.gz) <(zcat GCF_000366525.1_Bruc_abor_CNGB_1432_V1.genomic.fna.gz) <(zcat GCF_000250775.1_ASM25077v2.genomic.fna.gz) <(zcat GCF_004168605.1_ASM416860v1.genomic.fna.gz) <(zcat GCF_000008145.1_ASM814v1.genomic.fna.gz) <(zcat GCF_016484145.1_ASM1648414v1.genomic.fna.gz) <(zcat GCF_017177485.1_ASM1717748v1.genomic.fna.gz) <(zcat GCF_000369905.1_Bruc_abor_225_65_V1.genomic.fna.gz) <(zcat GCF_008932495.1_ASM893249v1.genomic.fna.gz) <(zcat GCF_017177365.1_ASM1717736v1.genomic.fna.gz) <(zcat GCF_000366105.1_Bruc_abor_65_110_V1.genomic.fna.gz) <(zcat GCF_000988815.1_BMEL001-SEQ-1-ASM-1.genomic.fna.gz) <(zcat GCF_000366885.1_Bruc_meli_CNGB_1120_V1.genomic.fna.gz) <(zcat GCF_008932875.1_ASM893287v1.genomic.fna.gz) <(zcat GCF_000366705.1_Bruc_abor_NI639_V1.genomic.fna.gz) <(zcat GCF_002979215.1_ASM297921v1.genomic.fna.gz) <(zcat GCF_000371145.1_Bruc_suis_CNGB_247_V1.genomic.fna.gz) <(zcat GCF_000007125.1_ASM712v1.genomic.fna.gz) <(zcat GCF_003989955.1_ASM398995v1.genomic.fna.gz) <(zcat GCF_001302545.1_ASM130254v1.genomic.fna.gz) <(zcat GCF_001715425.1_ASM171542v1.genomic.fna.gz) <(zcat GCF_003993825.1_ASM399382v1.genomic.fna.gz) <(zcat GCF_016126115.1_ASM1612611v1.genomic.fna.gz) <(zcat GCF_018138325.1_bwa-0.7.17.tar.bz2.genomic.fna.gz) <(zcat GCF_006505995.1_ASM650599v1.genomic.fna.gz) <(zcat GCF_002245555.1_ASM224555v1.genomic.fna.gz) <(zcat GCF_006507225.1_ASM650722v1.genomic.fna.gz) <(zcat GCF_000182725.1_ASM18272v1.genomic.fna.gz) <(zcat GCF_002192035.1_ASM219203v1.genomic.fna.gz) <(zcat GCF_002979295.1_ASM297929v1.genomic.fna.gz) <(zcat GCF_000740195.1_ASM74019v1.genomic.fna.gz) <(zcat GCF_008932635.1_ASM893263v1.genomic.fna.gz) <(zcat GCF_002278035.1_ASM227803v1.genomic.fna.gz) <(zcat GCF_000691585.1_ASM69158v1.genomic.fna.gz) <(zcat GCF_009665015.1_ASM966501v1.genomic.fna.gz) <(zcat GCF_003194895.1_ASM319489v1.genomic.fna.gz) <(zcat GCF_000223195.1_ASM22319v1.genomic.fna.gz) <(zcat GCF_002278675.1_ASM227867v1.genomic.fna.gz) <(zcat GCF_021165815.1_ASM2116581v1.genomic.fna.gz) <(zcat GCF_000371105.1_Bruc_suis_63_198_V1.genomic.fna.gz) <(zcat GCF_000298635.1_V1.0_of_Brucella_abortus_bv.1_str_134.genomic.fna.gz) <(zcat GCF_000479975.1_Bruc_meli_02-7258_V1.genomic.fna.gz) <(zcat GCF_000755085.1_ASM75508v1.genomic.fna.gz) <(zcat GCF_947242635.1_B122.genomic.fna.gz) <(zcat GCF_023651155.1_ASM2365115v1.genomic.fna.gz) <(zcat GCF_000365565.1_Bruc_suis_F8_06-2_V1.genomic.fna.gz) <(zcat GCF_002191915.1_ASM219191v1.genomic.fna.gz) <(zcat GCF_000245835.1_Bruc_abor_bv1_NI435a_V1.genomic.fna.gz) <(zcat GCF_017177415.1_ASM1717741v1.genomic.fna.gz) <(zcat GCF_002191695.1_ASM219169v1.genomic.fna.gz) <(zcat GCF_009789295.1_ASM978929v1.genomic.fna.gz) <(zcat GCF_000480235.1_Bruc_abor_07-0994-2411_V1.genomic.fna.gz) <(zcat GCF_002192055.1_ASM219205v1.genomic.fna.gz) <(zcat GCF_017139975.1_ASM1713997v1.genomic.fna.gz) <(zcat GCF_006507165.1_ASM650716v1.genomic.fna.gz) <(zcat GCF_023651075.1_ASM2365107v1.genomic.fna.gz) <(zcat GCF_002291345.1_ASM229134v1.genomic.fna.gz) <(zcat GCF_900236485.1_Bru_genome24.genomic.fna.gz) <(zcat GCF_001401915.1_ASM140191v1.genomic.fna.gz) <(zcat GCF_002290685.1_ASM229068v1.genomic.fna.gz) <(zcat GCF_004127195.1_ASM412719v1.genomic.fna.gz) <(zcat GCF_001856265.1_ASM185626v1.genomic.fna.gz) <(zcat GCF_013375275.1_ASM1337527v1.genomic.fna.gz) <(zcat GCF_001702375.1_ASM170237v1.genomic.fna.gz) <(zcat GCF_000480195.1_Bruc_sp_04-5288_V1.genomic.fna.gz) <(zcat GCF_003290345.1_ASM329034v1.genomic.fna.gz) <(zcat GCF_001866335.1_ASM186633v1.genomic.fna.gz) <(zcat GCF_023651805.1_ASM2365180v1.genomic.fna.gz) <(zcat GCF_001917355.1_ASM191735v1.genomic.fna.gz) <(zcat GCF_900446145.1_59071_D01.genomic.fna.gz) <(zcat GCF_028129085.1_ASM2812908v1.genomic.fna.gz) <(zcat GCF_017177855.2_Bm-501.genomic.fna.gz) <(zcat GCF_000366065.1_Bruc_ovis_IntaBari-2006-46-348_V1.genomic.fna.gz) <(zcat GCF_001940595.1_ASM194059v1.genomic.fna.gz) <(zcat GCF_006507445.1_ASM650744v1.genomic.fna.gz) <(zcat GCF_900236405.1_Bru_genome13.genomic.fna.gz) <(zcat GCF_013425785.1_ASM1342578v1.genomic.fna.gz) <(zcat GCF_009733205.1_Brucella_abortus_264.genomic.fna.gz) <(zcat GCF_002278575.1_ASM227857v1.genomic.fna.gz) <(zcat GCF_023651135.1_ASM2365113v1.genomic.fna.gz) <(zcat GCF_001307445.1_ASM130744v1.genomic.fna.gz) <(zcat GCF_000298575.1_V1.0_for_Brucella_canis_str_118.genomic.fna.gz) <(zcat GCF_002192135.1_ASM219213v1.genomic.fna.gz) <(zcat GCF_014199265.1_ASM1419926v1.genomic.fna.gz) <(zcat GCF_947241855.1_B123.genomic.fna.gz) <(zcat GCF_001078335.1_B.canis_SCL.genomic.fna.gz) <(zcat GCF_026256595.1_ASM2625659v1.genomic.fna.gz) <(zcat GCF_000022625.1_ASM2262v1.genomic.fna.gz) <(zcat GCF_003244395.3_ASM324439v3.genomic.fna.gz) <(zcat GCF_025908475.1_ASM2590847v1.genomic.fna.gz) <(zcat GCF_900446155.1_59302_G01.genomic.fna.gz) <(zcat GCF_001401935.1_ASM140193v1.genomic.fna.gz) <(zcat GCF_002191655.1_ASM219165v1.genomic.fna.gz) <(zcat GCF_022538045.1_ASM2253804v1.genomic.fna.gz) <(zcat GCF_000369985.1_Bruc_abor_63_130_V1.genomic.fna.gz) <(zcat GCF_000369925.1_Bruc_abor_355_78_V1.genomic.fna.gz) <(zcat GCF_000022745.1_ASM2274v1.genomic.fna.gz) <(zcat GCF_000662035.1_BrucellaCeti1.0.genomic.fna.gz) <(zcat GCF_003965935.1_ASM396593v1.genomic.fna.gz) <(zcat GCF_000366805.1_Bruc_abor_NI649_V1.genomic.fna.gz) <(zcat GCF_003984755.1_ASM398475v1.genomic.fna.gz) <(zcat GCF_006506545.1_ASM650654v1.genomic.fna.gz) <(zcat GCF_000018725.1_ASM1872v1.genomic.fna.gz) <(zcat GCF_006507905.1_ASM650790v1.genomic.fna.gz) <(zcat GCF_003244395.1_ASM324439v1.genomic.fna.gz) <(zcat GCF_000370785.1_Bruc_meli_Uk24_06_V1.genomic.fna.gz) <(zcat GCF_023651535.1_ASM2365153v1.genomic.fna.gz) <(zcat GCF_006517295.1_ASM651729v1.genomic.fna.gz) <(zcat GCF_003989885.1_ASM398988v1.genomic.fna.gz) <(zcat GCF_000740255.1_ASM74025v1.genomic.fna.gz) <(zcat GCF_020524305.1_ASM2052430v1.genomic.fna.gz) <(zcat GCF_004208675.1_ASM420867v1.genomic.fna.gz) <(zcat GCF_006476605.1_ASM647660v1.genomic.fna.gz) <(zcat GCF_002252475.1_ASM225247v1.genomic.fna.gz) <(zcat GCF_000158735.1_ASM15873v1.genomic.fna.gz) <(zcat GCF_947242175.1_B77.genomic.fna.gz) <(zcat GCF_000158675.1_ASM15867v1.genomic.fna.gz) <(zcat GCF_000480135.1_Bruc_suis_06-997-1672_V1.genomic.fna.gz) <(zcat GCF_023651655.1_ASM2365165v1.genomic.fna.gz) <(zcat GCF_016126195.1_ASM1612619v1.genomic.fna.gz) <(zcat GCF_000366665.1_Bruc_abor_NI628_V1.genomic.fna.gz) <(zcat GCF_000158695.1_ASM15869v1.genomic.fna.gz) <(zcat GCF_000366865.1_Bruc_meli_66_59_V1.genomic.fna.gz) <(zcat GCF_002191855.1_ASM219185v1.genomic.fna.gz) <(zcat GCF_000590795.1_ASM59079v1.genomic.fna.gz) <(zcat GCF_002290805.1_ASM229080v1.genomic.fna.gz) <(zcat GCF_006508005.1_ASM650800v1.genomic.fna.gz) <(zcat GCF_002191355.1_ASM219135v1.genomic.fna.gz) <(zcat GCF_000157775.1_ASM15777v1.genomic.fna.gz) <(zcat GCF_900236455.1_Bru_genome44.genomic.fna.gz) <(zcat GCF_006506435.1_ASM650643v1.genomic.fna.gz) <(zcat GCF_023554105.1_ASM2355410v1.genomic.fna.gz) <(zcat GCF_003194765.1_ASM319476v1.genomic.fna.gz) <(zcat GCF_002191675.1_ASM219167v1.genomic.fna.gz) <(zcat GCF_028620965.1_ASM2862096v1.genomic.fna.gz) <(zcat GCF_004704905.2_ASM470490v2.genomic.fna.gz) <(zcat GCF_947242095.1_B135.genomic.fna.gz) <(zcat GCF_000365945.1_Bruc_ovis_IntaBari-2001-319-5096_V1.genomic.fna.gz) <(zcat GCF_023651695.1_ASM2365169v1.genomic.fna.gz) <(zcat GCF_000370805.1_Bruc_meli_UK29_05_V1.genomic.fna.gz) <(zcat GCF_902825315.1_OCH-ISR4.genomic.fna.gz) <(zcat GCF_002291325.1_ASM229132v1.genomic.fna.gz) <(zcat GCF_947242465.1_B22.genomic.fna.gz) <(zcat GCF_000370945.1_Bruc_sp_63_311_V1.genomic.fna.gz) <(zcat GCF_000370345.1_Bruc_abor_88_217_V1.genomic.fna.gz) <(zcat GCF_028129055.1_ASM2812905v1.genomic.fna.gz) <(zcat GCF_023704275.1_ASM2370427v1.genomic.fna.gz) <(zcat GCF_000705095.1_Brucella_melitensis_C-554.genomic.fna.gz) <(zcat GCF_000370485.1_Bruc_abor_F6_05-9_V1.genomic.fna.gz) <(zcat GCF_000366185.1_Bruc_abor_NI380_V1.genomic.fna.gz) <(zcat GCF_000365685.1_Bruc_abor_NI388_V1.genomic.fna.gz) <(zcat GCF_000480095.1_Bruc_meli_02-5863-1_V1.genomic.fna.gz) <(zcat GCF_016126095.1_ASM1612609v1.genomic.fna.gz) <(zcat GCF_002266765.1_ASM226676v1.genomic.fna.gz) <(zcat GCF_006507285.1_ASM650728v1.genomic.fna.gz) <(zcat GCF_006507795.1_ASM650779v1.genomic.fna.gz) <(zcat GCF_001575075.1_ASM157507v1.genomic.fna.gz) <(zcat GCF_000366305.1_Bruc_abor_67_781_V1.genomic.fna.gz) <(zcat GCF_000366925.1_Bruc_meli_F10_05-2_V1.genomic.fna.gz) <(zcat GCF_008932475.1_ASM893247v1.genomic.fna.gz) <(zcat GCF_009017395.1_ASM901739v1.genomic.fna.gz) <(zcat GCF_000365885.1_Bruc_ovis_IntaBari-2006-46-332_V1.genomic.fna.gz) <(zcat GCF_003994435.1_ASM399443v1.genomic.fna.gz) <(zcat GCF_003856415.1_ASM385641v1.genomic.fna.gz) <(zcat GCF_001702255.1_ASM170225v1.genomic.fna.gz) <(zcat GCF_000478665.1_Bruc_abor_biovar_1_80-1399_V1.genomic.fna.gz) <(zcat GCF_947242015.1_B69.genomic.fna.gz) <(zcat GCF_023130505.1_ASM2313050v1.genomic.fna.gz) <(zcat GCF_001715405.1_ASM171540v1.genomic.fna.gz) <(zcat GCF_008932745.1_ASM893274v1.genomic.fna.gz) <(zcat GCF_947242135.1_B20.genomic.fna.gz) <(zcat GCF_002191255.1_ASM219125v1.genomic.fna.gz) <(zcat GCF_001307475.2_ASM130747v2.genomic.fna.gz) <(zcat GCF_006506735.1_ASM650673v1.genomic.fna.gz) <(zcat GCF_028620865.1_ASM2862086v1.genomic.fna.gz) <(zcat GCF_000699025.1_ASM69902v1.genomic.fna.gz) <(zcat GCF_001938175.1_ASM193817v1.genomic.fna.gz) <(zcat GCF_009664935.1_ASM966493v1.genomic.fna.gz) <(zcat GCF_000701065.1_Bru1330.genomic.fna.gz) <(zcat GCF_009733275.1_Brucella_abortus_401.genomic.fna.gz) <(zcat GCF_009789365.1_ASM978936v1.genomic.fna.gz) <(zcat GCF_001856295.1_ASM185629v1.genomic.fna.gz) <(zcat GCF_947242805.1_B6.genomic.fna.gz) <(zcat GCF_947242585.1_B35.genomic.fna.gz) <(zcat GCF_003049685.1_ASM304968v2.genomic.fna.gz) <(zcat GCF_000530495.1_Oliveri.genomic.fna.gz) <(zcat GCF_000292085.1_ASM29208v2.genomic.fna.gz) <(zcat GCF_000479955.1_Bruc_abor_01-4165_V1.genomic.fna.gz) <(zcat GCF_947242105.1_B132.genomic.fna.gz) <(zcat GCF_004208655.1_ASM420865v1.genomic.fna.gz) <(zcat GCF_947242285.1_B55.genomic.fna.gz) <(zcat GCF_000398805.1_Bruc_abor_93_2_V1.genomic.fna.gz) <(zcat GCF_023651115.1_ASM2365111v1.genomic.fna.gz) <(zcat GCF_947242215.1_B145.genomic.fna.gz) <(zcat GCF_006506425.1_ASM650642v1.genomic.fna.gz) <(zcat GCF_008932485.1_ASM893248v1.genomic.fna.gz) <(zcat GCF_017177485.2_Bm-510.genomic.fna.gz) <(zcat GCF_000209635.2_ASM20963v2.genomic.fna.gz) <(zcat GCF_947241755.1_B13.genomic.fna.gz) <(zcat GCF_003937425.1_ASM393742v1.genomic.fna.gz) <(zcat GCF_023651175.1_ASM2365117v1.genomic.fna.gz) <(zcat GCF_947241985.1_B139.genomic.fna.gz) <(zcat GCF_002252445.1_ASM225244v1.genomic.fna.gz) <(zcat GCF_024970345.1_ASM2497034v1.genomic.fna.gz) <(zcat GCF_023651615.1_ASM2365161v1.genomic.fna.gz) <(zcat GCF_002291245.1_ASM229124v1.genomic.fna.gz) <(zcat GCF_002191335.1_ASM219133v1.genomic.fna.gz) <(zcat GCF_002191935.1_ASM219193v1.genomic.fna.gz) <(zcat GCF_016411965.1_ASM1641196v1.genomic.fna.gz) <(zcat GCF_900236545.1_Bru_genome36.genomic.fna.gz) <(zcat GCF_017177315.1_ASM1717731v1.genomic.fna.gz) <(zcat GCF_009733215.1_Brucella_abortus_293.genomic.fna.gz) <(zcat GCF_002192015.1_ASM219201v1.genomic.fna.gz) <(zcat GCF_000600055.1_ASM60005v1.genomic.fna.gz) <(zcat GCF_947242505.1_B113.genomic.fna.gz) <(zcat GCF_002191995.1_ASM219199v1.genomic.fna.gz) <(zcat GCF_017177615.2_ASM1717761v2.genomic.fna.gz) <(zcat GCF_000348645.1_MD5v1.genomic.fna.gz) <(zcat GCF_002191755.1_ASM219175v1.genomic.fna.gz) <(zcat GCF_006517305.1_ASM651730v1.genomic.fna.gz) <(zcat GCF_002204265.1_ASM220426v1.genomic.fna.gz) <(zcat GCF_002979335.1_ASM297933v1.genomic.fna.gz) <(zcat GCF_947241765.1_B80.genomic.fna.gz) <(zcat GCF_900236415.1_Bru_genome48.genomic.fna.gz) <(zcat GCF_000370305.1_Bruc_abor_87_28_V1.genomic.fna.gz) <(zcat GCF_011801185.1_ASM1180118v1.genomic.fna.gz) <(zcat GCF_947241945.1_B126.genomic.fna.gz) <(zcat GCF_900236345.1_Bru_genome47.genomic.fna.gz) <(zcat GCF_008932625.1_ASM893262v1.genomic.fna.gz) <(zcat GCF_003991385.1_ASM399138v1.genomic.fna.gz) <(zcat GCF_002290745.1_ASM229074v1.genomic.fna.gz) <(zcat GCF_008932905.1_ASM893290v1.genomic.fna.gz) <(zcat GCF_002291205.1_ASM229120v1.genomic.fna.gz) <(zcat GCF_000365925.1_Bruc_ovis_IntaBari-2002-82-58_V1.genomic.fna.gz) <(zcat GCF_000366225.1_Bruc_suis_92_63_V1.genomic.fna.gz) <(zcat GCF_006517315.1_ASM651731v1.genomic.fna.gz) <(zcat GCF_024970895.1_ASM2497089v1.genomic.fna.gz) <(zcat GCF_947242675.1_B2.genomic.fna.gz) <(zcat GCF_900446025.1_58932_E01.genomic.fna.gz) <(zcat GCF_006505895.1_ASM650589v1.genomic.fna.gz) <(zcat GCF_000370185.1_Bruc_abor_78_14_V1.genomic.fna.gz) <(zcat GCF_000370245.1_Bruc_abor_80_28_V1.genomic.fna.gz) <(zcat GCF_006506855.1_ASM650685v1.genomic.fna.gz) <(zcat GCF_000292165.1_ASM29216v2.genomic.fna.gz) <(zcat GCF_001766425.1_ASM176642v1.genomic.fna.gz) <(zcat GCF_000371005.1_Bruc_sp_F8_99_V1.genomic.fna.gz) <(zcat GCF_000209615.2_ASM20961v2.genomic.fna.gz) <(zcat GCF_001856185.1_ASM185618v1.genomic.fna.gz) <(zcat GCF_000365965.1_Bruc_ovis_IntaBari-2001-319-4082_V1.genomic.fna.gz) <(zcat GCF_947242255.1_B150.genomic.fna.gz) <(zcat GCF_947241865.1_B15.genomic.fna.gz) <(zcat GCF_002192175.1_ASM219217v1.genomic.fna.gz) <(zcat GCF_000444515.1_Denovo_assembly.genomic.fna.gz) <(zcat GCF_000365825.1_Bruc_abor_NI492_V1.genomic.fna.gz) <(zcat GCF_008932785.1_ASM893278v1.genomic.fna.gz) <(zcat GCF_002191795.1_ASM219179v1.genomic.fna.gz) <(zcat GCF_000366325.1_Bruc_abor_78_36_V1.genomic.fna.gz) <(zcat GCF_900491925.1_10469.genomic.fna.gz) <(zcat GCF_000370605.1_Bruc_cani_CNGB_513_V1.genomic.fna.gz) <(zcat GCF_000371345.1_Bruc_ovis_IntaBari-2010-47-268_V1.genomic.fna.gz) <(zcat GCF_002192075.1_ASM219207v1.genomic.fna.gz) <(zcat GCF_006507425.1_ASM650742v1.genomic.fna.gz) <(zcat GCF_002191275.1_ASM219127v1.genomic.fna.gz) <(zcat GCF_947242835.1_B159.genomic.fna.gz) <(zcat GCF_000366285.1_Bruc_abor_64_122_V1.genomic.fna.gz) <(zcat GCF_009664965.1_ASM966496v1.genomic.fna.gz) <(zcat GCF_000480055.1_Bruc_suis_04-0115_V1.genomic.fna.gz) <(zcat GCF_006507815.1_ASM650781v1.genomic.fna.gz) <(zcat GCF_000370585.1_Bruc_cani_79_122_V1.genomic.fna.gz) <(zcat GCF_000413675.1_Bruc_abor_biovar_4_90-0775_V1.genomic.fna.gz) <(zcat GCF_000740335.1_ASM74033v1.genomic.fna.gz) <(zcat GCF_023650995.1_ASM2365099v1.genomic.fna.gz) <(zcat GCF_000740355.1_ASM74035v1.genomic.fna.gz) <(zcat GCF_947242305.1_B66.genomic.fna.gz) <(zcat GCF_026256925.1_ASM2625692v1.genomic.fna.gz) <(zcat GCF_001302525.1_ASM130252v1.genomic.fna.gz) <(zcat GCF_000157835.1_ASM15783v1.genomic.fna.gz) <(zcat GCF_003993855.1_ASM399385v1.genomic.fna.gz) <(zcat GCF_000365645.1_Bruc_abor_levi_gila_V1.genomic.fna.gz) <(zcat GCF_000371325.1_Bruc_suis_F9_06-1_V1.genomic.fna.gz) <(zcat GCF_004137325.1_ASM413732v1.genomic.fna.gz) <(zcat GCF_002290765.1_ASM229076v1.genomic.fna.gz) <(zcat GCF_000480215.1_Bruc_meli_11-1823-3434_V1.genomic.fna.gz) <(zcat GCF_006507685.1_ASM650768v1.genomic.fna.gz) <(zcat GCF_006507125.1_ASM650712v1.genomic.fna.gz) <(zcat GCF_003993875.1_ASM399387v1.genomic.fna.gz) <(zcat GCF_000157675.1_ASM15767v1.genomic.fna.gz) <(zcat GCF_002150355.1_ASM215035v1.genomic.fna.gz) <(zcat GCF_026256565.1_ASM2625656v1.genomic.fna.gz) <(zcat GCF_000192885.1_ASM19288v1.genomic.fna.gz) <(zcat GCF_006507825.1_ASM650782v1.genomic.fna.gz) <(zcat GCF_003989905.1_ASM398990v1.genomic.fna.gz) <(zcat GCF_000250815.1_ASM25081v2.genomic.fna.gz) <(zcat GCF_003994015.1_ASM399401v1.genomic.fna.gz) <(zcat GCF_003010715.1_ASM301071v1.genomic.fna.gz) <(zcat GCF_000698325.1_ASM69832v1.genomic.fna.gz) <(zcat GCF_023651675.1_ASM2365167v1.genomic.fna.gz) <(zcat GCF_947242885.1_B134.genomic.fna.gz) <(zcat GCF_000740235.1_ASM74023v1.genomic.fna.gz) <(zcat GCF_000742215.1_ASM74221v1.genomic.fna.gz) <(zcat GCF_002191435.1_ASM219143v1.genomic.fna.gz) <(zcat GCF_000480155.1_Bruc_suis_97-9757_V1.genomic.fna.gz) <(zcat GCF_001856305.1_ASM185630v1.genomic.fna.gz) <(zcat GCF_000370425.1_Bruc_abor_F5_04-7_V1.genomic.fna.gz) <(zcat GCF_002290665.1_ASM229066v1.genomic.fna.gz) <(zcat GCF_006507855.1_ASM650785v1.genomic.fna.gz) <(zcat GCF_024802385.1_ASM2480238v1.genomic.fna.gz) <(zcat GCF_000366005.1_Bruc_ovis_IntaBari-1993-758_V1.genomic.fna.gz) <(zcat GCF_000371285.1_Bruc_suis_F8_06-1_V1.genomic.fna.gz) <(zcat GCF_000366485.1_Bruc_abor_93_1_V1.genomic.fna.gz) <(zcat GCF_900236425.1_Bru_genome11.genomic.fna.gz) <(zcat GCF_027625455.1_ASM2762545v1.genomic.fna.gz) <(zcat GCF_900236565.1_Bru_genome8.genomic.fna.gz) <(zcat GCF_001758265.1_ASM175826v1.genomic.fna.gz) <(zcat GCF_008728025.1_ASM872802v1.genomic.fna.gz) <(zcat GCF_003989715.1_ASM398971v1.genomic.fna.gz) <(zcat GCF_947242125.1_B56.genomic.fna.gz) <(zcat GCF_947241975.1_B155.genomic.fna.gz) <(zcat GCF_002204345.1_ASM220434v1.genomic.fna.gz) <(zcat GCF_001608425.1_BMEL003-SEQ-1-ASM-1.genomic.fna.gz) <(zcat GCF_002191735.1_ASM219173v1.genomic.fna.gz) <(zcat GCF_009733455.1_Brucella_abortus_I-29.genomic.fna.gz) <(zcat GCF_016126215.1_ASM1612621v1.genomic.fna.gz) <(zcat GCF_017177735.1_ASM1717773v1.genomic.fna.gz) <(zcat GCF_900236555.1_Bru_genome5.genomic.fna.gz) <(zcat GCF_947242085.1_B109.genomic.fna.gz) <(zcat GCF_009728185.1_ASM972818v1.genomic.fna.gz) <(zcat GCF_000370045.1_Bruc_abor_63_168_V1.genomic.fna.gz) <(zcat GCF_000365725.1_Bruc_abor_F3_07-1_V1.genomic.fna.gz) <(zcat GCF_026256505.1_ASM2625650v1.genomic.fna.gz) <(zcat GCF_000157875.1_ASM15787v1.genomic.fna.gz) <(zcat GCF_000413515.1_Bruc_ovis_IntaBari-2009-88-3_V1.genomic.fna.gz) <(zcat GCF_000370885.1_Bruc_ovis_63_96_V1.genomic.fna.gz) <(zcat GCF_006506835.1_ASM650683v1.genomic.fna.gz) <(zcat GCF_947242055.1_B149.genomic.fna.gz) <(zcat GCF_006506675.1_ASM650667v1.genomic.fna.gz) <(zcat GCF_902825355.1_OCH-ISR3.genomic.fna.gz) <(zcat GCF_002266745.1_ASM226674v1.genomic.fna.gz) <(zcat GCF_000182625.1_ASM18262v1.genomic.fna.gz) <(zcat GCF_009789215.1_ASM978921v1.genomic.fna.gz) <(zcat GCF_900446035.1_59071_C01.genomic.fna.gz) <(zcat GCF_000370865.1_Bruc_meli_UK37_05_V1.genomic.fna.gz) <(zcat GCF_000292045.1_ASM29204v1.genomic.fna.gz) <(zcat GCF_900236585.1_Bru_genome1.genomic.fna.gz) <(zcat GCF_014138165.1_ASM1413816v1.genomic.fna.gz) <(zcat GCF_003194885.1_ASM319488v1.genomic.fna.gz) <(zcat GCF_947242665.1_B116.genomic.fna.gz) <(zcat GCF_006506115.1_ASM650611v1.genomic.fna.gz) <(zcat GCF_006507665.1_ASM650766v1.genomic.fna.gz) <(zcat GCF_947242265.1_B78.genomic.fna.gz) <(zcat GCF_002191515.1_ASM219151v1.genomic.fna.gz) <(zcat GCF_947242145.1_B75.genomic.fna.gz) <(zcat GCF_023651635.1_ASM2365163v1.genomic.fna.gz) <(zcat GCF_006506965.1_ASM650696v1.genomic.fna.gz) <(zcat GCF_002310355.1_ASM231035v1.genomic.fna.gz) <(zcat GCF_003993955.1_ASM399395v1.genomic.fna.gz) <(zcat GCF_003994045.1_ASM399404v1.genomic.fna.gz) <(zcat GCF_947241795.1_B158.genomic.fna.gz) <(zcat GCF_900243435.1_ASM90024343v1.genomic.fna.gz) <(zcat GCF_006506875.1_ASM650687v1.genomic.fna.gz) <(zcat GCF_002191615.1_ASM219161v1.genomic.fna.gz) <(zcat GCF_026256545.1_ASM2625654v1.genomic.fna.gz) <(zcat GCF_000413635.1_Bruc_abor_biovar_1_90-0962_V1.genomic.fna.gz) <(zcat GCF_000480295.1_Bruc_cani_04-2330-1_V1.genomic.fna.gz) <(zcat GCF_000366125.1_Bruc_abor_CNGB_752_V1.genomic.fna.gz) <(zcat GCF_014884485.1_ASM1488448v1.genomic.fna.gz) <(zcat GCF_023651515.1_ASM2365151v1.genomic.fna.gz) <(zcat GCF_000163135.1_ASM16313v1.genomic.fna.gz) <(zcat GCF_001856195.1_ASM185619v1.genomic.fna.gz) <(zcat GCF_001267725.1_ASM126772v1.genomic.fna.gz) <(zcat GCF_000245935.1_Bruc_abor_bv1_NI016_V1.genomic.fna.gz) <(zcat GCF_023651215.1_ASM2365121v1.genomic.fna.gz) <(zcat GCF_900236575.1_Bru_genome10.genomic.fna.gz) <(zcat GCF_000365585.1_Bruc_suis_F5_03-2_V1.genomic.fna.gz) <(zcat GCF_004168645.1_ASM416864v1.genomic.fna.gz) <(zcat GCF_023651255.1_ASM2365125v1.genomic.fna.gz) <(zcat GCF_009733335.1_Brucella_abortus_1552.genomic.fna.gz) <(zcat GCF_000365705.1_Bruc_suis_F4_06-146_V1.genomic.fna.gz) <(zcat GCF_023093775.1_ASM2309377v1.genomic.fna.gz) <(zcat GCF_947242755.1_B121.genomic.fna.gz) <(zcat GCF_000480015.1_Bruc_abor_90-12178_V1.genomic.fna.gz) <(zcat GCF_000370705.1_Bruc_meli_F9_05_V1.genomic.fna.gz) <(zcat GCF_006506505.1_ASM650650v1.genomic.fna.gz) <(zcat GCF_000158775.1_ASM15877v1.genomic.fna.gz) <(zcat GCF_028621395.1_ASM2862139v1.genomic.fna.gz) <(zcat GCF_016483975.1_ASM1648397v1.genomic.fna.gz) <(zcat GCF_000371265.1_Bruc_suis_F7_06-5_V1.genomic.fna.gz) <(zcat GCF_002252505.1_ASM225250v1.genomic.fna.gz) <(zcat GCF_000245895.1_Bruc_abor_bv1_NI488_V1.genomic.fna.gz) <(zcat GCF_014138145.1_ASM1413814v1.genomic.fna.gz) <(zcat GCF_016483915.1_ASM1648391v1.genomic.fna.gz) <(zcat GCF_000472245.1_Denovo_assembly.genomic.fna.gz) <(zcat GCF_000875695.1_ASM87569v1.genomic.fna.gz) <(zcat GCF_000251205.1_ASM25120v1.genomic.fna.gz) <(zcat GCF_006506925.1_ASM650692v1.genomic.fna.gz) <(zcat GCF_001401945.1_ASM140194v1.genomic.fna.gz) <(zcat GCF_009789305.1_ASM978930v1.genomic.fna.gz) <(zcat GCF_002191895.1_ASM219189v1.genomic.fna.gz) <(zcat GCF_002290785.1_ASM229078v1.genomic.fna.gz) <(zcat GCF_947242745.1_B129.genomic.fna.gz) <(zcat GCF_003994025.1_ASM399402v1.genomic.fna.gz) <(zcat GCF_002191235.1_ASM219123v1.genomic.fna.gz) <(zcat GCF_000018905.1_ASM1890v1.genomic.fna.gz) <(zcat GCF_018798825.1_ASM1879882v1.genomic.fna.gz) <(zcat GCF_001938135.1_ASM193813v1.genomic.fna.gz) <(zcat GCF_000365765.1_Bruc_abor_F6_05-2_V1.genomic.fna.gz) <(zcat GCF_003244395.2_ASM324439v2.genomic.fna.gz) <(zcat GCF_000370125.1_Bruc_abor_65_157_V1.genomic.fna.gz) <(zcat GCF_017140095.1_ASM1714009v1.genomic.fna.gz) <(zcat GCF_004704905.1_ASM470490v1.genomic.fna.gz) <(zcat GCF_001431745.1_ASM143174v1.genomic.fna.gz) <(zcat GCF_002278735.1_ASM227873v1.genomic.fna.gz) <(zcat GCF_013409355.1_ASM1340935v1.genomic.fna.gz) <(zcat GCF_006506785.1_ASM650678v1.genomic.fna.gz) <(zcat GCF_016126175.1_ASM1612617v1.genomic.fna.gz) <(zcat GCF_000714755.1_Brucella_melitensis_C-558.genomic.fna.gz) <(zcat GCF_000413615.1_Bruc_abor_biovar_1_87-0095_V1.genomic.fna.gz) <(zcat GCF_000157715.1_ASM15771v1.genomic.fna.gz) <(zcat GCF_008932275.1_ASM893227v1.genomic.fna.gz) <(zcat GCF_000366645.1_Bruc_abor_NI518_V1.genomic.fna.gz) <(zcat GCF_000413535.1_Bruc_abor_biovar_4_68-3396P_V1.genomic.fna.gz) <(zcat GCF_000366585.1_Bruc_meli_UK22_06_V1.genomic.fna.gz) <(zcat GCF_008801705.1_ASM880170v1.genomic.fna.gz) <(zcat GCF_001641495.1_ASM164149v1.genomic.fna.gz) <(zcat GCF_003516085.1_ASM351608v1.genomic.fna.gz) <(zcat GCF_002291285.1_ASM229128v1.genomic.fna.gz) <(zcat GCF_000157815.1_ASM15781v1.genomic.fna.gz) <(zcat GCF_000367085.1_Bruc_ovis_F8_05B_V1.genomic.fna.gz) <(zcat GCF_000366985.1_Bruc_meli_F5_07-239A_V1.genomic.fna.gz) <(zcat GCF_000480075.1_Bruc_abor_03-4923-239-D_V1.genomic.fna.gz) <(zcat GCF_000016845.1_ASM1684v1.genomic.fna.gz) <(zcat GCF_016126245.1_ASM1612624v1.genomic.fna.gz) <(zcat GCF_014193785.1_ASM1419378v1.genomic.fna.gz) <(zcat GCF_900460605.1_58116_D02.genomic.fna.gz) <(zcat GCF_008932295.1_ASM893229v1.genomic.fna.gz) <(zcat GCF_000413835.1_Bruc_abor_biovar_1_B10-0973_V1.genomic.fna.gz) <(zcat GCF_023651095.1_ASM2365109v1.genomic.fna.gz) <(zcat GCF_008932685.1_ASM893268v1.genomic.fna.gz) <(zcat GCF_947242365.1_B151.genomic.fna.gz) <(zcat GCF_006506205.1_ASM650620v1.genomic.fna.gz) <(zcat GCF_947242075.1_B110.genomic.fna.gz) <(zcat GCF_947242005.1_B111.genomic.fna.gz) <(zcat GCF_006507345.1_ASM650734v1.genomic.fna.gz) <(zcat GCF_006506625.1_ASM650662v1.genomic.fna.gz) <(zcat GCF_947242575.1_B104.genomic.fna.gz) <(zcat GCF_000740155.1_ASM74015v1.genomic.fna.gz) <(zcat GCF_002953595.1_ASM295359v1.genomic.fna.gz) <(zcat GCF_000209595.2_ASM20959v2.genomic.fna.gz) <(zcat GCF_000739325.1_ASM73932v1.genomic.fna.gz) <(zcat GCF_002192095.1_ASM219209v1.genomic.fna.gz) <(zcat GCF_023651845.1_ASM2365184v1.genomic.fna.gz) <(zcat GCF_000245915.1_Bruc_abor_bv1_NI010_V1.genomic.fna.gz) <(zcat GCF_002150445.1_ASM215044v1.genomic.fna.gz) <(zcat GCF_000755105.1_ASM75510v1.genomic.fna.gz) <(zcat GCF_009665065.1_ASM966506v1.genomic.fna.gz) <(zcat GCF_947242875.1_B157.genomic.fna.gz) <(zcat GCF_001302485.1_ASM130248v1.genomic.fna.gz) <(zcat GCF_009733365.1_Brucella_abortus_C-550.genomic.fna.gz) <(zcat GCF_000182645.1_ASM18264v1.genomic.fna.gz) <(zcat GCF_006507915.1_ASM650791v1.genomic.fna.gz) <(zcat GCF_000366605.1_Bruc_abor_63_59_V1.genomic.fna.gz) <(zcat GCF_000740275.1_ASM74027v1.genomic.fna.gz) <(zcat GCF_003325675.1_ASM332567v1.genomic.fna.gz) <(zcat GCF_947242605.1_B81.genomic.fna.gz) <(zcat GCF_947241875.1_B12.genomic.fna.gz) <(zcat GCF_000298615.1_V1.0_of_Brucella_melitensis_bv.3_str_128.genomic.fna.gz) <(zcat GCF_015326295.1_ASM1532629v1.genomic.fna.gz) <(zcat GCF_001637305.1_ASM163730v1.genomic.fna.gz) <(zcat GCF_008932555.1_ASM893255v1.genomic.fna.gz) <(zcat GCF_016887905.1_ASM1688790v1.genomic.fna.gz) <(zcat GCF_017797885.1_ASM1779788v1.genomic.fna.gz) <(zcat GCF_002278715.1_ASM227871v1.genomic.fna.gz) <(zcat GCF_000018525.1_ASM1852v1.genomic.fna.gz) <(zcat GCF_000160255.1_ASM16025v1.genomic.fna.gz) <(zcat GCF_003993935.1_ASM399393v1.genomic.fna.gz) <(zcat GCF_947241915.1_B48.genomic.fna.gz) <(zcat GCF_006506235.1_ASM650623v1.genomic.fna.gz) <(zcat GCF_002191715.1_ASM219171v1.genomic.fna.gz) <(zcat GCF_000157855.1_ASM15785v1.genomic.fna.gz) <(zcat GCF_016126075.1_ASM1612607v1.genomic.fna.gz) <(zcat GCF_000366165.1_Bruc_abor_CNGB_966_V1.genomic.fna.gz) <(zcat GCF_003966035.1_ASM396603v1.genomic.fna.gz) <(zcat GCF_000740215.1_ASM74021v1.genomic.fna.gz) <(zcat GCF_002191535.1_ASM219153v1.genomic.fna.gz) <(zcat GCF_907163095.1_OB_assembly.genomic.fna.gz) <(zcat GCF_000742955.1_ASM74295v1.genomic.fna.gz) <(zcat GCF_023651395.1_ASM2365139v1.genomic.fna.gz) <(zcat GCF_013087535.1_ASM1308753v1.genomic.fna.gz) <(zcat GCF_000413735.1_Bruc_abor_biovar_1_94-1313_V1.genomic.fna.gz) <(zcat GCF_900417225.1_00-2529-3.genomic.fna.gz) <(zcat GCF_000182425.1_ASM18242v1.genomic.fna.gz) <(zcat GCF_002262955.1_ASM226295v1.genomic.fna.gz) <(zcat GCF_002803535.1_ASM280353v1.genomic.fna.gz) <(zcat GCF_902825345.1_OCH-ISR5.genomic.fna.gz) <(zcat GCF_002191815.1_ASM219181v1.genomic.fna.gz) <(zcat GCF_900236365.1_Bru_genome40.genomic.fna.gz) <(zcat GCF_009728165.1_B_abortus_str_240.genomic.fna.gz) <(zcat GCF_000366685.1_Bruc_abor_NI593_V1.genomic.fna.gz) <(zcat GCF_947242355.1_B152.genomic.fna.gz) <(zcat GCF_001937795.1_ASM193779v1.genomic.fna.gz) <(zcat GCF_000366745.1_Bruc_abor_NI613_V1.genomic.fna.gz) <(zcat GCF_014884545.1_ASM1488454v1.genomic.fna.gz) <(zcat GCF_000938035.1_Brucella_melitensis_Phil1136_12.genomic.fna.gz) <(zcat GCF_000370765.1_Bruc_meli_UK23_06_V1.genomic.fna.gz) <(zcat GCF_028620645.1_ASM2862064v1.genomic.fna.gz) <(zcat GCF_002191635.1_ASM219163v1.genomic.fna.gz) <(zcat GCF_003664555.1_ASM366455v1.genomic.fna.gz) <(zcat GCF_017177695.1_ASM1717769v1.genomic.fna.gz) <(zcat GCF_008932725.1_ASM893272v1.genomic.fna.gz) <(zcat GCF_001652485.1_ASM165248v1.genomic.fna.gz) <(zcat GCF_002150395.1_ASM215039v1.genomic.fna.gz) <(zcat GCF_006506605.1_ASM650660v1.genomic.fna.gz) <(zcat GCF_000292145.1_ASM29214v2.genomic.fna.gz) <(zcat GCF_000370905.1_Bruc_ovis_81_8_V1.genomic.fna.gz) <(zcat GCF_000366505.1_Bruc_abor_CNGB_1011_V1.genomic.fna.gz) <(zcat GCF_000479995.1_Bruc_abor_03-2770-11_V1.genomic.fna.gz) <(zcat GCF_900417195.1_01-3016-5171.genomic.fna.gz) <(zcat GCF_000741995.1_ASM74199v1.genomic.fna.gz) <(zcat GCF_008932765.1_ASM893276v1.genomic.fna.gz) <(zcat GCF_000413595.1_Bruc_abor_biovar_1_85-1058_V1.genomic.fna.gz) <(zcat GCF_000480115.1_Bruc_abor_99-9971-159_V1.genomic.fna.gz) <(zcat GCF_000366765.1_Bruc_abor_NI633_V1.genomic.fna.gz) <(zcat GCF_904066045.1_WS-Be-68.genomic.fna.gz) <(zcat GCF_009789335.1_ASM978933v1.genomic.fna.gz) <(zcat GCF_006508035.1_ASM650803v1.genomic.fna.gz) <(zcat GCF_947241925.1_B51.genomic.fna.gz) <(zcat GCF_017177535.1_ASM1717753v1.genomic.fna.gz) <(zcat GCF_008932575.1_ASM893257v1.genomic.fna.gz) <(zcat GCF_000367125.1_Bruc_sp_UK38_05_V1.genomic.fna.gz) <(zcat GCF_947242065.1_B65.genomic.fna.gz) <(zcat GCF_000370625.1_Bruc_meli_BG2_S27_V1.genomic.fna.gz) <(zcat GCF_000017405.1_ASM1740v1.genomic.fna.gz) <(zcat GCF_006507065.1_ASM650706v1.genomic.fna.gz) <(zcat GCF_014884405.1_ASM1488440v1.genomic.fna.gz) <(zcat GCF_017177825.2_Bm-498.genomic.fna.gz) <(zcat GCF_000054005.1_ASM5400v1.genomic.fna.gz) <(zcat GCF_006506565.1_ASM650656v1.genomic.fna.gz) <(zcat GCF_000370745.1_Bruc_meli_UK22_04_V1.genomic.fna.gz) <(zcat GCF_900491895.1_07-2859-6071.genomic.fna.gz) <(zcat GCF_000250835.1_ASM25083v2.genomic.fna.gz) <(zcat GCF_025490555.1_ASM2549055v1.genomic.fna.gz) <(zcat GCF_000370725.1_Bruc_meli_UK14_06_V1.genomic.fna.gz) <(zcat GCF_000366425.1_Bruc_abor_88_19_V1.genomic.fna.gz) <(zcat GCF_009733465.1_Brucella_abortus_I-34.genomic.fna.gz) <(zcat GCF_006507335.1_ASM650733v1.genomic.fna.gz) <(zcat GCF_006507745.1_ASM650774v1.genomic.fna.gz) <(zcat GCF_002994775.1_ASM299477v1.genomic.fna.gz) <(zcat GCF_900417215.1_00-4216.genomic.fna.gz) <(zcat GCF_002191415.1_ASM219141v1.genomic.fna.gz) <(zcat GCF_000705115.1_Brucella_melitensis_C-555.genomic.fna.gz) <(zcat GCF_900446125.1_57572_C01.genomic.fna.gz) <(zcat GCF_000366345.1_Bruc_abor_80_102_V1.genomic.fna.gz) <(zcat GCF_000371165.1_Bruc_suis_F12_02_V1.genomic.fna.gz) <(zcat GCF_000740945.1_ASM74094v1.genomic.fna.gz) <(zcat GCF_001402005.1_ASM140200v1.genomic.fna.gz) <(zcat GCF_017177875.1_ASM1717787v1.genomic.fna.gz) <(zcat GCF_002263015.1_ASM226301v1.genomic.fna.gz) <(zcat GCF_004137225.1_ASM413722v1.genomic.fna.gz) <(zcat GCF_006506005.1_ASM650600v1.genomic.fna.gz) <(zcat GCF_000158795.1_ASM15879v1.genomic.fna.gz) <(zcat GCF_017177675.1_ASM1717767v1.genomic.fna.gz) <(zcat GCF_017177415.2_Bm-512.genomic.fna.gz) <(zcat GCF_900236525.1_Bru_genome6.genomic.fna.gz) <(zcat GCF_009733495.1_Brucella_abortus_I-181.genomic.fna.gz) <(zcat GCF_947242485.1_B79.genomic.fna.gz) <(zcat GCF_014084005.1_ASM1408400v1.genomic.fna.gz) <(zcat GCF_006506035.1_ASM650603v1.genomic.fna.gz) <(zcat GCF_000740315.1_ASM74031v1.genomic.fna.gz) <(zcat GCF_000612245.1_OchInt1.0.genomic.fna.gz) <(zcat GCF_003994095.1_ASM399409v1.genomic.fna.gz) <(zcat GCF_000370365.1_Bruc_abor_F1_06-B21_V1.genomic.fna.gz) <(zcat GCF_000245955.1_Bruc_abor_bv1_NI021_V1.genomic.fna.gz) <(zcat GCF_002191455.1_ASM219145v1.genomic.fna.gz) <(zcat GCF_006507105.1_ASM650710v1.genomic.fna.gz) <(zcat GCF_017177715.1_ASM1717771v1.genomic.fna.gz) <(zcat GCF_001937875.1_ASM193787v1.genomic.fna.gz) <(zcat GCF_002191575.1_ASM219157v1.genomic.fna.gz) <(zcat GCF_000371205.1_Bruc_suis_F5_05-4_V1.genomic.fna.gz) <(zcat GCF_017177745.1_ASM1717774v1.genomic.fna.gz) <(zcat GCF_009733375.1_Brucella_abortus_C-551.genomic.fna.gz) <(zcat GCF_003989725.1_ASM398972v1.genomic.fna.gz) <(zcat GCF_900000005.1_BVF60.genomic.fna.gz) <(zcat GCF_947242345.1_B32.genomic.fna.gz) <(zcat GCF_000413975.1_Bruc_abor_biovar_1_89-0363_V1.genomic.fna.gz) <(zcat GCF_000413555.1_Bruc_abor_biovar_2_82-3893_V1.genomic.fna.gz) <(zcat GCF_000371025.1_Bruc_sp_F96_2_V1.genomic.fna.gz) <(zcat GCF_000292125.1_ASM29212v1.genomic.fna.gz) <(zcat GCF_001938045.1_ASM193804v1.genomic.fna.gz) <(zcat GCF_028621035.1_ASM2862103v1.genomic.fna.gz) <(zcat GCF_023651315.1_ASM2365131v1.genomic.fna.gz) <(zcat GCF_002262975.1_ASM226297v1.genomic.fna.gz) <(zcat GCF_009733285.1_Brucella_abortus_403.genomic.fna.gz) <(zcat GCF_024970695.1_ASM2497069v1.genomic.fna.gz) <(zcat GCF_001857585.1_ASM185758v1.genomic.fna.gz) <(zcat GCF_017177615.1_ASM1717761v1.genomic.fna.gz) <(zcat GCF_008932285.1_ASM893228v1.genomic.fna.gz) <(zcat GCF_024539785.1_ASM2453978v1.genomic.fna.gz) <(zcat GCF_006505935.1_ASM650593v1.genomic.fna.gz) <(zcat GCF_001766475.1_ASM176647v1.genomic.fna.gz) <(zcat GCF_000366145.1_Bruc_abor_CNGB_759_V1.genomic.fna.gz) <(zcat GCF_001715505.1_ASM171550v1.genomic.fna.gz) <(zcat GCF_023108915.1_ASM2310891v1.genomic.fna.gz) <(zcat GCF_002278695.1_ASM227869v1.genomic.fna.gz) <(zcat GCF_026241335.1_ASM2624133v1.genomic.fna.gz) <(zcat GCF_001937915.1_ASM193791v1.genomic.fna.gz) <(zcat GCF_000367065.1_Bruc_ovis_80_125_V1.genomic.fna.gz) <(zcat GCF_006506135.1_ASM650613v1.genomic.fna.gz) <(zcat GCF_006507295.1_ASM650729v1.genomic.fna.gz) <(zcat GCF_000292185.1_ASM29218v2.genomic.fna.gz) <(zcat GCF_006507325.1_ASM650732v1.genomic.fna.gz) <(zcat GCF_000755125.1_ASM75512v1.genomic.fna.gz) <(zcat GCF_002895105.1_ASM289510v1.genomic.fna.gz) <(zcat GCF_900497665.1_09-369-776-1.genomic.fna.gz) <(zcat GCF_016126105.1_ASM1612610v1.genomic.fna.gz) <(zcat GCF_902825335.1_OCH-ISR1.genomic.fna.gz) <(zcat GCF_003989635.1_ASM398963v1.genomic.fna.gz) <(zcat GCF_017177855.1_ASM1717785v1.genomic.fna.gz) <(zcat GCF_000740135.1_ASM74013v1.genomic.fna.gz) <(zcat GCF_003994185.1_ASM399418v1.genomic.fna.gz) <(zcat GCF_016483965.1_ASM1648396v1.genomic.fna.gz) <(zcat GCF_002150345.1_ASM215034v1.genomic.fna.gz) <(zcat GCF_000366845.1_Bruc_meli_64_150_V1.genomic.fna.gz) <(zcat GCF_002266775.1_ASM226677v1.genomic.fna.gz) <(zcat GCF_003994175.1_ASM399417v1.genomic.fna.gz) <(zcat GCF_001702315.1_ASM170231v1.genomic.fna.gz) <(zcat GCF_000739335.1_ASM73933v1.genomic.fna.gz) <(zcat GCF_000370505.1_Bruc_abor_NI352_V1.genomic.fna.gz) <(zcat GCF_000365625.1_Bruc_abor_LEVI237_V1.genomic.fna.gz) <(zcat GCF_002290125.1_ASM229012v1.genomic.fna.gz) <(zcat GCF_006507135.1_ASM650713v1.genomic.fna.gz) <(zcat GCF_000409625.1_60a.genomic.fna.gz) <(zcat GCF_947242445.1_B16.genomic.fna.gz) <(zcat GCF_000366545.1_Bruc_abor_CNGB_308_V1.genomic.fna.gz) <(zcat GCF_009789395.1_ASM978939v1.genomic.fna.gz) <(zcat GCF_009789345.1_ASM978934v1.genomic.fna.gz) <(zcat GCF_006506515.1_ASM650651v1.genomic.fna.gz) <(zcat GCF_900236435.1_Bru_genome23.genomic.fna.gz) <(zcat GCF_000366785.1_Bruc_abor_NI645_V1.genomic.fna.gz) <(zcat GCF_000250795.1_ASM25079v2.genomic.fna.gz) <(zcat GCF_008761615.1_ASM876161v1.genomic.fna.gz) <(zcat GCF_000365745.1_Bruc_abor_F3_01-0300_V1.genomic.fna.gz) <(zcat GCF_000331655.1_Brucella_melitensis_043.genomic.fna.gz) <(zcat GCF_000238175.1_ASM23817v1.genomic.fna.gz) <(zcat GCF_002252535.1_ASM225253v1.genomic.fna.gz) <(zcat GCF_016484065.1_ASM1648406v1.genomic.fna.gz) <(zcat GCF_000370265.1_Bruc_abor_84_26_V1.genomic.fna.gz) <(zcat GCF_000157755.1_ASM15775v1.genomic.fna.gz) <(zcat GCF_001937825.1_ASM193782v1.genomic.fna.gz) <(zcat GCF_000365845.1_Bruc_meli_F15_06-7_V1.genomic.fna.gz) <(zcat GCF_000365605.1_Bruc_abor_F2_06-8_V1.genomic.fna.gz) <(zcat GCF_017177815.2_Bm-500.genomic.fna.gz) <(zcat GCF_023651235.1_ASM2365123v1.genomic.fna.gz) <(zcat GCF_000298595.1_V1.0_of_Brucella_melitensis_bv_1_str_133.genomic.fna.gz) <(zcat GCF_000370085.1_Bruc_abor_64_108_V1.genomic.fna.gz) <(zcat GCF_006507595.1_ASM650759v1.genomic.fna.gz) <(zcat GCF_947241805.1_B14.genomic.fna.gz) <(zcat GCF_016128015.1_ASM1612801v1.genomic.fna.gz) <(zcat GCF_000160295.1_ASM16029v1.genomic.fna.gz) <(zcat GCF_001971625.1_ASM197162v1.genomic.fna.gz) <(zcat GCF_023651435.1_ASM2365143v1.genomic.fna.gz) <(zcat GCF_947242815.1_B53.genomic.fna.gz) <(zcat GCF_947242845.1_B38.genomic.fna.gz) <(zcat GCF_023651375.1_ASM2365137v1.genomic.fna.gz) <(zcat GCF_902807045.1_17-2122-4144.genomic.fna.gz) <(zcat GCF_000662035.2_Bceti2.0.genomic.fna.gz) <(zcat GCF_009733175.1_Brucella_abortus_317.genomic.fna.gz) <(zcat GCF_000366205.1_Bruc_suis_92_29_V1.genomic.fna.gz) <(zcat GCF_000366625.1_Bruc_meli_CNGB_1076_V1.genomic.fna.gz) <(zcat GCF_009789465.1_ASM978946v1.genomic.fna.gz) <(zcat GCF_006507075.1_ASM650707v1.genomic.fna.gz) <(zcat GCF_001715485.1_ASM171548v1.genomic.fna.gz) <(zcat GCF_000367265.1_Bruc_cani_CNGB_1172_V1.genomic.fna.gz) <(zcat GCF_003516045.1_ASM351604v1.genomic.fna.gz) <(zcat GCF_900236465.1_Bru_genome45.genomic.fna.gz) <(zcat GCF_000371085.1_Bruc_suis_01-5744_V1.genomic.fna.gz) <(zcat GCF_900417245.1_00-3099-5734.genomic.fna.gz) <(zcat GCF_003984745.1_ASM398474v1.genomic.fna.gz) <(zcat GCF_947242555.1_B108.genomic.fna.gz) <(zcat GCF_000698345.1_ASM69834v1.genomic.fna.gz) <(zcat GCF_000332835.1_ASM33283v1.genomic.fna.gz) <(zcat GCF_009789415.1_ASM978941v1.genomic.fna.gz) <(zcat GCF_006505925.1_ASM650592v1.genomic.fna.gz) <(zcat GCF_000158715.1_ASM15871v1.genomic.fna.gz) <(zcat GCF_020526685.1_ASM2052668v1.genomic.fna.gz) <(zcat GCF_947242515.1_B118.genomic.fna.gz) <(zcat GCF_006507365.1_ASM650736v1.genomic.fna.gz) <(zcat GCF_004127225.1_ASM412722v1.genomic.fna.gz) <(zcat GCF_008932585.1_ASM893258v1.genomic.fna.gz) <(zcat GCF_016107505.1_ASM1610750v1.genomic.fna.gz) <(zcat GCF_000662015.1_BrucellaInopinata1.0.genomic.fna.gz) <(zcat GCF_000413775.1_Bruc_abor_biovar_1_01-0585_V1.genomic.fna.gz) <(zcat GCF_003194875.1_ASM319487v1.genomic.fna.gz) <(zcat GCF_015624465.1_ASM1562446v1.genomic.fna.gz) <(zcat GCF_000163115.1_ASM16311v1.genomic.fna.gz) <(zcat GCF_947241965.1_B125.genomic.fna.gz) <(zcat GCF_000370965.1_Bruc_sp_F23_97_V1.genomic.fna.gz) <(zcat GCF_947242565.1_B107.genomic.fna.gz) <(zcat GCF_009789445.1_ASM978944v1.genomic.fna.gz) <(zcat GCF_947242045.1_B142.genomic.fna.gz) <(zcat GCF_023651455.1_ASM2365145v1.genomic.fna.gz) <(zcat GCF_003994205.1_ASM399420v1.genomic.fna.gz) <(zcat GCF_900095085.1_14913_1_10.genomic.fna.gz) <(zcat GCF_000366265.1_Bruc_suis_CNGB_786_V1.genomic.fna.gz) <(zcat GCF_003432005.1_ASM343200v1.genomic.fna.gz) <(zcat GCF_900236375.1_Bru_genome9.genomic.fna.gz) <(zcat GCF_017177445.1_ASM1717744v1.genomic.fna.gz) <(zcat GCF_000413755.1_Bruc_abor_biovar_1_01-0648_V1.genomic.fna.gz) <(zcat GCF_002191315.1_ASM219131v1.genomic.fna.gz) <(zcat GCF_000413715.1_Bruc_abor_biovar_2_90-1280_V1.genomic.fna.gz) <(zcat GCF_002204335.1_ASM220433v1.genomic.fna.gz) <(zcat GCF_001594165.1_ASM159416v1.genomic.fna.gz) <(zcat GCF_900236385.1_Bru_genome37.genomic.fna.gz) <(zcat GCF_006507675.1_ASM650767v1.genomic.fna.gz) <(zcat GCF_947242315.1_B54.genomic.fna.gz) <(zcat GCF_000367285.1_Bruc_cani_CNGB_1324_V1.genomic.fna.gz) <(zcat GCF_024126535.1_ASM2412653v1.genomic.fna.gz) <(zcat GCF_017177395.1_ASM1717739v1.genomic.fna.gz) <(zcat GCF_006506015.1_ASM650601v1.genomic.fna.gz) <(zcat GCF_000331635.1_Brucella_019.genomic.fna.gz) <(zcat GCF_006506095.1_ASM650609v1.genomic.fna.gz) <(zcat GCF_001307475.1_ASM130747v1.genomic.fna.gz) <(zcat GCF_006507185.1_ASM650718v1.genomic.fna.gz) <(zcat GCF_017177745.2_ASM1717774v2.genomic.fna.gz) <(zcat GCF_947241905.1_B40.genomic.fna.gz) <(zcat GCF_001856285.1_ASM185628v1.genomic.fna.gz) <(zcat GCF_947242785.1_B21.genomic.fna.gz) <(zcat GCF_002871075.1_ASM287107v1.genomic.fna.gz) <(zcat GCF_002191375.1_ASM219137v1.genomic.fna.gz) <(zcat GCF_014884525.1_ASM1488452v1.genomic.fna.gz) <(zcat GCF_000480035.1_Bruc_suis_06-791-1309_V1.genomic.fna.gz) <(zcat GCF_000661995.1_BrucellaSuis1.0.genomic.fna.gz) <(zcat GCF_900446115.1_59302_H01.genomic.fna.gz) <(zcat GCF_014270005.1_ASM1427000v1.genomic.fna.gz) <(zcat GCF_000585655.1_Ochant_v5.genomic.fna.gz) <(zcat GCF_013109225.1_ASM1310922v1.genomic.fna.gz) <(zcat GCF_012395285.1_ASM1239528v1.genomic.fna.gz) <(zcat GCF_000370285.1_Bruc_abor_85_69_V1.genomic.fna.gz) <(zcat GCF_002763615.1_ASM276361v1.genomic.fna.gz) <(zcat GCF_904066075.1_WS-Be-61.genomic.fna.gz) <(zcat GCF_000740375.1_ASM74037v1.genomic.fna.gz) <(zcat GCF_900446105.1_59302_A02.genomic.fna.gz) <(zcat GCF_002975235.1_ASM297523v1.genomic.fna.gz) <(zcat GCF_000477635.1_Bruc_abor_BC95_V1.genomic.fna.gz) <(zcat GCF_000367025.1_Bruc_meli_R3_07-2_V1.genomic.fna.gz) <(zcat GCF_023651595.1_ASM2365159v1.genomic.fna.gz) <(zcat GCF_000370845.1_Bruc_meli_UK31_99_V1.genomic.fna.gz) <(zcat GCF_016091965.1_ASM1609196v1.genomic.fna.gz) <(zcat GCF_001402075.1_ASM140207v1.genomic.fna.gz) <(zcat GCF_006506935.1_ASM650693v1.genomic.fna.gz) <(zcat GCF_000366825.1_Bruc_cani_UK10_02_V1.genomic.fna.gz) <(zcat GCF_006506195.1_ASM650619v1.genomic.fna.gz) <(zcat GCF_000365805.1_Bruc_abor_NI274_V1.genomic.fna.gz) <(zcat GCF_000182235.1_ASM18223v1.genomic.fna.gz) <(zcat GCF_006507495.1_ASM650749v1.genomic.fna.gz) <(zcat GCF_028129065.1_ASM2812906v1.genomic.fna.gz) <(zcat GCF_001402095.1_ASM140209v1.genomic.fna.gz) <(zcat GCF_001401995.1_ASM140199v1.genomic.fna.gz) <(zcat GCF_006506975.1_ASM650697v1.genomic.fna.gz) <(zcat GCF_903131625.1_BCH.genomic.fna.gz) <(zcat GCF_947242195.1_B17.genomic.fna.gz) <(zcat GCF_947241935.1_B131.genomic.fna.gz) <(zcat GCF_009733195.1_Brucella_abortus_313.genomic.fna.gz) <(zcat GCF_000370685.1_Bruc_meli_F8_01-155_V1.genomic.fna.gz) <(zcat GCF_947241845.1_B161.genomic.fna.gz) <(zcat GCF_000370525.1_Bruc_abor_NI422_V1.genomic.fna.gz) <(zcat GCF_000365865.1_Bruc_meli_B115_V1.genomic.fna.gz) <(zcat GCF_003965945.1_ASM396594v1.genomic.fna.gz) <(zcat GCF_023651495.1_ASM2365149v1.genomic.fna.gz) <(zcat GCF_947242695.1_B39.genomic.fna.gz) <(zcat GCF_001715385.1_ASM171538v1.genomic.fna.gz) <(zcat GCF_000661995.2_Bsuis2.0.genomic.fna.gz) <(zcat GCF_006506665.1_ASM650666v1.genomic.fna.gz) <(zcat GCF_017177815.1_ASM1717781v1.genomic.fna.gz) <(zcat GCF_002191555.1_ASM219155v1.genomic.fna.gz) <(zcat GCF_006506635.1_ASM650663v1.genomic.fna.gz) <(zcat GCF_000007505.1_ASM750v1.genomic.fna.gz) <(zcat GCF_002252525.1_ASM225252v1.genomic.fna.gz) <(zcat GCF_000366565.1_Bruc_abor_CNGB_436_V1.genomic.fna.gz) <(zcat GCF_008932835.1_ASM893283v1.genomic.fna.gz) <(zcat GCF_003194805.1_ASM319480v1.genomic.fna.gz) <(zcat GCF_006506815.1_ASM650681v1.genomic.fna.gz) <(zcat GCF_000365785.1_Bruc_abor_F1_06-B1_V1.genomic.fna.gz) <(zcat GCF_016806105.1_ASM1680610v1.genomic.fna.gz) <(zcat GCF_900446135.1_58116_C01.genomic.fna.gz) <(zcat GCF_900417235.1_00-2782.genomic.fna.gz) <(zcat GCF_003993995.1_ASM399399v1.genomic.fna.gz) <(zcat GCF_006506025.1_ASM650602v1.genomic.fna.gz) <(zcat GCF_000370985.1_Bruc_sp_F5_06_V1.genomic.fna.gz) <(zcat GCF_000227645.1_ASM22764v1.genomic.fna.gz) <(zcat GCF_000209575.2_ASM20957v2.genomic.fna.gz) <(zcat GCF_006517335.1_ASM651733v1.genomic.fna.gz) <(zcat GCF_000158755.1_ASM15875v1.genomic.fna.gz) <(zcat GCF_006506105.1_ASM650610v1.genomic.fna.gz) <(zcat GCF_001715365.1_ASM171536v1.genomic.fna.gz) <(zcat GCF_008932885.1_ASM893288v1.genomic.fna.gz) <(zcat GCF_001715465.1_ASM171546v1.genomic.fna.gz) <(zcat GCF_002291385.1_ASM229138v1.genomic.fna.gz) <(zcat GCF_009823695.1_ASM982369v1.genomic.fna.gz) <(zcat GCF_006507465.1_ASM650746v1.genomic.fna.gz) <(zcat GCF_009789435.1_ASM978943v1.genomic.fna.gz) <(zcat GCF_000366365.1_Bruc_abor_80_108_V1.genomic.fna.gz) <(zcat GCF_000370145.1_Bruc_abor_65_63_V1.genomic.fna.gz) <(zcat GCF_947242435.1_B24.genomic.fna.gz) <(zcat GCF_000480175.1_Bruc_abor_89-2646-1238_V1.genomic.fna.gz) <(zcat GCF_947241825.1_B127.genomic.fna.gz) <(zcat GCF_947242735.1_B138.genomic.fna.gz) <(zcat GCF_002290705.1_ASM229070v1.genomic.fna.gz) <(zcat GCF_006506685.1_ASM650668v1.genomic.fna.gz) <(zcat GCF_000480255.1_Bruc_abor_99-9971-135_V1.genomic.fna.gz) <(zcat GCF_900417205.1_00-2956-4.genomic.fna.gz) <(zcat GCF_016091955.1_ASM1609195v1.genomic.fna.gz) <(zcat GCF_000370445.1_Bruc_abor_F6_05-3_V1.genomic.fna.gz) <(zcat GCF_003194825.1_ASM319482v1.genomic.fna.gz) <(zcat GCF_003994075.1_ASM399407v1.genomic.fna.gz) <(zcat GCF_024799785.1_ASM2479978v1.genomic.fna.gz) <(zcat GCF_004208695.1_ASM420869v1.genomic.fna.gz) <(zcat GCF_001608355.1_BMEL002-SEQ-1-ASM-1.genomic.fna.gz) <(zcat GCF_900454235.1_34576_A02.genomic.fna.gz) <(zcat GCF_000366045.1_Bruc_ovis_IntaBari-2009-88-4_V1.genomic.fna.gz) <(zcat GCF_006506895.1_ASM650689v1.genomic.fna.gz) <(zcat GCF_006507875.1_ASM650787v1.genomic.fna.gz) <(zcat GCF_002191875.1_ASM219187v1.genomic.fna.gz) <(zcat GCF_900446055.1_59014_A01.genomic.fna.gz) <(zcat GCF_947241815.1_B162.genomic.fna.gz) <(zcat GCF_014495905.1_ASM1449590v1.genomic.fna.gz) <(zcat GCF_002262995.1_ASM226299v1.genomic.fna.gz) <(zcat GCF_028620895.1_ASM2862089v1.genomic.fna.gz) <(zcat GCF_014495725.1_ASM1449572v1.genomic.fna.gz) <(zcat GCF_017177895.1_ASM1717789v1.genomic.fna.gz) <(zcat GCF_009017355.1_ASM901735v1.genomic.fna.gz) <(zcat GCF_947241835.1_B160.genomic.fna.gz) <(zcat GCF_900236445.1_Bru_genome2.genomic.fna.gz) <(zcat GCF_000370005.1_Bruc_abor_63_138_V1.genomic.fna.gz) <(zcat GCF_000739315.1_ASM73931v1.genomic.fna.gz) <(zcat GCF_001937805.1_ASM193780v1.genomic.fna.gz) <(zcat GCF_000370565.1_Bruc_abor_R42-08_V1.genomic.fna.gz) <(zcat GCF_001766405.1_ASM176640v1.genomic.fna.gz) <(zcat GCF_000370165.1_Bruc_abor_67_93_V1.genomic.fna.gz) <(zcat GCF_006508015.1_ASM650801v1.genomic.fna.gz) <(zcat GCF_000370825.1_Bruc_meli_UK3_06_V1.genomic.fna.gz) <(zcat GCF_000698245.1_ASM69824v1.genomic.fna.gz) <(zcat GCF_003993895.1_ASM399389v1.genomic.fna.gz) <(zcat GCF_023651035.1_ASM2365103v1.genomic.fna.gz) <(zcat GCF_017177355.1_ASM1717735v1.genomic.fna.gz) <(zcat GCF_002749475.1_ASM274947v1.genomic.fna.gz) 

In [281]:
  <(zcat GCF_005938105.1_ASM593810v1.genomic.fna.gz)

SyntaxError: invalid syntax (166177143.py, line 1)

### count individual files and dump to tsv

In [117]:
cd data

/hobbitholes/waymoot/hchen/acheron/acheron/data


In [46]:
'''# first file test
file = open("jellyfishcmd.sh", "w")
cmd = 'jellyfish count -m 25 -s 100M -t 10 -o firstoutput.jf -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz) \n jellyfish dump -L 25 -U 1139 firstoutput.jf> firstoutput.tsv'
file.write(cmd)
file.close()
file1 = open("jellyfishcmd.sh", "r")
file1.readlines()'''

['jellyfish count -m 25 -s 100M -t 10 -o firstoutput.jf -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz) \n',
 ' jellyfish dump -L 25 -U 1139 firstoutput.jf> firstoutput.tsv']

In [103]:
all_file_names[0:2]

['GCF_900446045.1_58932_D01.genomic.fna.gz',
 'GCF_005938105.1_ASM593810v1.genomic.fna.gz']

In [122]:
file0 = open("all_jellyfish_cmds.sh", "w")
for file_names in all_file_names:
    cmd = 'jellyfish count -m 25 -s 100M -t 10 -o ' + file_names[:-15] + '.jf -C <(zcat ' + file_names + ')\njellyfish dump -L 25 -U 1139 ' + file_names[:-15] + '.jf> ' + file_names[:-15] + '.tsv' +' \n'
    file0.write(cmd)
    
file0.close()
file01 = open("all_jellyfish_cmds.sh", "r")
file01.readlines()

['jellyfish count -m 25 -s 100M -t 10 -o GCF_900446045.1_58932_D01.jf -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz)\n',
 'jellyfish dump -L 25 -U 1139 GCF_900446045.1_58932_D01.jf> GCF_900446045.1_58932_D01.tsv \n',
 'jellyfish count -m 25 -s 100M -t 10 -o GCF_005938105.1_ASM593810v1.jf -C <(zcat GCF_005938105.1_ASM593810v1.genomic.fna.gz)\n',
 'jellyfish dump -L 25 -U 1139 GCF_005938105.1_ASM593810v1.jf> GCF_005938105.1_ASM593810v1.tsv \n',
 'jellyfish count -m 25 -s 100M -t 10 -o GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.jf -C <(zcat GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.genomic.fna.gz)\n',
 'jellyfish dump -L 25 -U 1139 GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.jf> GCF_000413955.1_Bruc_abor_biovar_1_B10-0091_V1.tsv \n',
 'jellyfish count -m 25 -s 100M -t 10 -o GCF_022024335.1_ASM2202433v1.jf -C <(zcat GCF_022024335.1_ASM2202433v1.genomic.fna.gz)\n',
 'jellyfish dump -L 25 -U 1139 GCF_022024335.1_ASM2202433v1.jf> GCF_022024335.1_ASM2202433v1.tsv \n',
 'jellyf

In [123]:
ls *.sh

all_jellyfish_cmds.sh*


#### in terminal:

In [ ]:
chmod +x all_jellyfish_cmds.sh

In [23]:
./all_jellyfish_cmds.sh

SyntaxError: invalid syntax (3082608780.py, line 1)

In [ ]:
sbatch -c 4 --mem 10G --wrap="./all_jellyfish_cmds.sh"

In [48]:
ls firstoutput*

firstoutput.jf  firstoutput.tsv


In [53]:
firstoutput_file1 = open("firstoutput.tsv")
firstoutput_contents1 = firstoutput_file1.readlines()
len(firstoutput_contents1)

62

In [51]:
firstoutput_file = open("firstoutput.tsv")
firstoutput_contents = firstoutput_file.readlines()
len(firstoutput_contents)

62

In [43]:
firstoutput=pd.read_csv('firstoutput.tsv',sep='\n', header = None, delimiter = ">")
firstoutput

,0,1
0,NaN,29.0
1,AGAGCGCATCCCGAAAAGTGTGAAA,NaN
2,NaN,31.0
3,GAGCGCATCCCGAAAAGTGTGAAAC,NaN
4,NaN,25.0
...,...,...
57,CCGTTTCACACTTTTCGGGATGCGC,NaN
58,NaN,28.0
59,GCATCTTTTCCGAAAACCGTTTCAC,NaN
60,NaN,33.0


In [ ]:
'''
for i in range(len(firstoutput)):
    count = []
    kmer = []
    if i % 2 ==0:
        count.append(firstoutput[0][i])
    elif i % 2 !=0:
        kmer.append(firstoutput[0][i])
kmer
'''

In [33]:
1%2

1

In [31]:
yes = []
yes.append(firstoutput[0][1])
yes

['AGAGCGCATCCCGAAAAGTGTGAAA']

### Sam's function

In [17]:
def read_to_csv(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        hk = []
        c = []
        #for line,hk,c in zip(lines, hashed_kmers, counts):
        for line in lines:
            col = line.split('\t')
            hk.append(mmh3.hash64(col[0],42)[0])
            c.append(col[1].strip('\n'))
        df = pd.DataFrame({file:c},
                            index=hk)
        df[file] = df[file].astype('int8')
        return df

In [20]:
files = [file for file in os.listdir('./') if file.endswith('.tsv')]
files

['firstoutput.tsv']

In [21]:
read_to_csv(files)

TypeError: expected str, bytes or os.PathLike object, not list

In [27]:
exp = open("firstoutput.jf", encoding = "utf-8")
contents = exp.readlines()
head(contents)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 1522: invalid start byte

In [ ]:
jellyfish dump firstoutput.jf > firstoutput1.jf

In [ ]:
--out-counter-len

In [98]:
1 % 2

1

In [88]:
lines[1].strip('\n')

'TTGCGGCGCTTCTCGAAGGCCGTCT'

In [ ]:
haha = open(r"mer_counts_dumps.jf",encoding="utf-8")
lines = haha.readlines()
counts = []
kmers = []
for i in range(len(lines)):
    if i % 2 == 0:
        counts.append(lines[i].strip('\n'))
    elif i % 2 == 1:
        kmers.append(lines[i].strip('\n'))
    else:
        print("oops!!!!!!!!!!!!!!!!!!!")
        break

In [120]:
len(lines)

6521234

In [ ]:
len(lines)

In [115]:
counts

['>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',

In [118]:
for i in range(len(counts)):
    counts[i] = counts[i][1:]

In [119]:
counts

['1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1'

In [108]:
counts

['>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',
 '>1',

In [15]:
def read_to_csv(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        hk = []
        c = []
        #for line,hk,c in zip(lines, hashed_kmers, counts):
        for line in lines:
            col = line.split('\t')
            hk.append(mmh3.hash64(col[0],42)[0])
            c.append(col[1].strip('\n'))
        df = pd.DataFrame({file:c},
                            index=hk)
        df[file] = df[file].astype('int8')
        return df

In [21]:
file = 'mer_counts_dumps.jf'
with open(file, 'r') as f:
    lines = f.readlines()
    hk = []
    c = []
    for line in lines:
        col = line.split('\t')
        hk.append(mmh3.hash64(col[0],42)[0])
        c.append(col[0].strip('\n'))
    df = pd.DataFrame({file:c},
                        index=hk)
#     df[file] = df[file].astype('int8')

In [28]:
df[df['mer_counts_dumps.jf'] != '>1']

,mer_counts_dumps.jf
-8076741129589595366,GCGGTGCAGCTTTGCAGCTTCATGA
1671319429224671905,AAGGCAATACACAGATGCTGCATTG
-603573424757275838,ATTGCATCCAGAACGGCCTTTTTGG
1964028542105443693,CGAAGGCAATGAACCGCTGAAAGAA
8684858065960153592,TTCGACGGCGACGACGAAGAGGAAA
...,...
4264416083999768166,GCAGGAGCATTCACTGATCTTCCAA
4800995997732918606,CGCCTCGATCTTCTCGCGCATTTCA
548695629886190444,CATCATGCCGTAGGAGGCAGGGCCA
-4473836802714327848,GAAATCCGCCGTGCCGGAAGTCACC


In [19]:
col[0]

'>1\n'

In [9]:
read_to_csv('mer_counts_dumps.jf')

IndexError: list index out of range

In [286]:
pd.read_csv('mer_counts_dumps.csv')

,>185
0,AAAAAAAAAAAAAAAAAAAAAAAAA
1,>1029
2,CCGGCGCATGGGCCGGGCTTTTCTC
3,>1
4,ATCGCAGGCATCGTTAATGGTGATC
...,...
336745656,GCGTTCTTGGTGAAACCGCTCACAC
336745657,>1
336745658,TCCTTTTCTATGGACCCGTAGCTCA
336745659,>2


In [275]:
cmd1 = 'gzip -dk GCF_000007505.1_ASM750v1.genomic.fna.gz'.split()
step1 = subprocess.run(cmd1, shell = False, check= False, stdout = subprocess.PIPE,
                           stderr = subprocess.PIPE)

In [276]:
gzip GCF_900446045.1_58932_D01.genomic.fna

SyntaxError: invalid syntax (258855515.py, line 1)

In [260]:
cmd1 = 'jellyfish count -m 25 -s 100M -t 10 -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz)'.split()
subprocess.run(cmd1, shell = False, check= False, stdout = subprocess.PIPE,
                           stderr = subprocess.PIPE)

CompletedProcess(args=['jellyfish', 'count', '-m', '25', '-s', '100M', '-t', '10', '-C', '<(zcat', 'GCF_900446045.1_58932_D01.genomic.fna.gz)'], returncode=-6, stdout=b'', stderr=b"terminate called after throwing an instance of 'std::runtime_error'\n  what():  Can't open file '<(zcat'\n")

In [264]:
ls | wc -l

1167


In [268]:
ls *.fna.gz

GCF_000007125.1_ASM712v1.genomic.fna.gz
GCF_000007505.1_ASM750v1.genomic.fna.gz
GCF_000008145.1_ASM814v1.genomic.fna.gz
GCF_000016845.1_ASM1684v1.genomic.fna.gz
GCF_000017405.1_ASM1740v1.genomic.fna.gz
GCF_000018525.1_ASM1852v1.genomic.fna.gz
GCF_000018725.1_ASM1872v1.genomic.fna.gz
GCF_000018905.1_ASM1890v1.genomic.fna.gz
GCF_000022625.1_ASM2262v1.genomic.fna.gz
GCF_000022745.1_ASM2274v1.genomic.fna.gz
GCF_000054005.1_ASM5400v1.genomic.fna.gz
GCF_000157675.1_ASM15767v1.genomic.fna.gz
GCF_000157695.1_ASM15769v1.genomic.fna.gz
GCF_000157715.1_ASM15771v1.genomic.fna.gz
GCF_000157735.1_ASM15773v1.genomic.fna.gz
GCF_000157755.1_ASM15775v1.genomic.fna.gz
GCF_000157775.1_ASM15777v1.genomic.fna.gz
GCF_000157795.1_ASM15779v1.genomic.fna.gz
GCF_000157815.1_ASM15781v1.genomic.fna.gz
GCF_000157835.1_ASM15783v1.genomic.fna.gz
GCF_000157855.1_ASM15785v1.genomic.fna.gz
GCF_000157875.1_ASM15787v1.genomic.fna.gz
GCF_000158675.1_ASM15867v1.genomic.fna.gz
GCF_000158695.1_ASM15869v1.genomic.fna.gz
GCF_00

In [259]:
cmd1 = 'ls'.split()
step1 = subprocess.run(cmd1, shell = False, check= False,stdout = subprocess.PIPE,
                       stderr = subprocess.PIPE)
cmd2 = 'xargs -n 1 echo gunzip -c > generators.txt'.split()
step2 = subprocess.run(cmd2, shell = False, check = False)

gunzip -c > generators


In [ ]:
stdout = subprocess.PIPE,
                       stderr = subprocess.PIPE

In [ ]:
jellyfish count -g generators -G 4 ...

In [ ]:
GCF_000007505.1_ASM750v1.genomic.fna.gz

In [ ]:
jellyfish count -m 25 -s 100M -t 10 -C 

In [ ]:
jellyfish count -m 4 -s 100M -t 10 -C <(zcat GCF_900446045.1_58932_D01.genomic.fna.gz)

In [ ]:
jellyfish dump mer_counts.jf > mer_counts_dumps.tsv

# Kmer counting using LOOP

In [4]:
def count_kmers(read, k):
    """Count kmer occurrences in a given read.

    Parameters
    ----------
    read : string
        A single DNA sequence.
    k : int
        The value of k for which to count kmers.

    Returns
    -------
    counts : dictionary, {'string': int}
        A dictionary of counts keyed by their individual kmers (strings
        of length k).

    Examples
    --------
    >>> count_kmers("GATGAT", 3)
    {'ATG': 1, 'GAT': 2, 'TGA': 1}
    """
    # Start with an empty dictionary
    counts = {}
    # Calculate how many kmers of length k there are
    num_kmers = len(read) - k + 1
    # Loop over the kmer start positions
    for i in range(num_kmers):
        # Slice the string to get the kmer
        kmer = read[i:i+k]
        # Add the kmer to the dictionary if it's not there
        if kmer not in counts:
            counts[kmer] = 0
        # Increment the count for this kmer
        counts[kmer] += 1
    # Return the final counts
    return counts

In [125]:
# experimental cell for 
pdd1 = pd.DataFrame([[1,2,2], [1,2,2]], columns = ['yes','no','haha'])
pdd2 = pd.DataFrame([[1,2,2],[10,15,18]], columns = ['no', 'yes', 'waha'])
# pd.concat([pdd1,pdd2], ignore_index = True)
print(pdd1)
print(pdd2)
merged = pd.merge(pdd1, pdd2, how = 'outer').fillna(0)
if np.shape(merged)[1]%2 != 0:
    print("you screwed up")
print(merged)
updated_pdd1 = merged.head(int(np.shape(merged)[1]/2)).reset_index(drop = True)
updated_pdd2 = merged.tail(int(np.shape(merged)[1]/2)).reset_index(drop = True)
merged.head(int(np.shape(merged)[1]/2)).reset_index(drop = True) + merged.tail(int(np.shape(merged)[1]/2)).reset_index(drop = True)
# updated_pdd1 + updated_pdd2

   yes  no  haha
0    1   2     2
1    1   2     2
   no  yes  waha
0   1    2     2
1  10   15    18
   yes  no  haha  waha
0    1   2   2.0   0.0
1    1   2   2.0   0.0
2    2   1   0.0   2.0
3   15  10   0.0  18.0


,yes,no,haha,waha
0,3,3,2.0,2.0
1,16,12,2.0,18.0


In [ ]:
k = 6
fastaFile = gzip.open(all_file_directories[0], "rt", encoding='utf-8')
print(fastaFile)
kmerCmd = '../kmer-counter --fasta --k=%d %s' % (k, fastaFile)
try:
    output = subprocess.check_output(kmerCmd, shell=True)
    print(output)
    result = {}
    for line in output.splitlines():
        (header, counts) = line.strip().split('\t')
        header = header[1:]
        kmers = dict((k,int(v)) for (k,v) in [d.split(':') for d in counts.split(' ')])
        result[header] = kmers
    sys.stdout.write("%s\n" % (str(result)))
except subprocess.CalledProcessError as error:
    sys.stderr.write("%s\n" % (str(error)))


In [216]:
%%time

final_df = pd.DataFrame([{}])
for file in all_file_directories[0:1]:
    with gzip.open(file, "rt") as unzipped_files:
        loop_count = 0
#         seq_list = []
        combine_df = pd.DataFrame([{}])
        for data in SeqIO.parse(unzipped_files, 'fasta'):
            df = pd.DataFrame([{'test': 0}])
            species = []
            kmers = ['test']
            counts = [0]
            loop_count += 1
#             print('now it is in loop: {}.'.format(loop_count))
            identifier = data.id
            description = data.description
            sequence = data.seq
#             print('Processing the record {}:'.format(identifier))
#             species = description.split()[1:3].join()
#             print('Its description is: \n{}'.format(description))
#             amount_of_nucleotides = len(sequence)
#             print('Its sequence contains {} nucleotides.'.format(amount_of_nucleotides))
            seq_list = list(sequence)
            sequence_data = ''.join(seq_list)
#             print(count_kmers(sequence_data, 25))
            for key, value in count_kmers(sequence_data, 25).items():
                kmers.append(key)
                counts.append(value)
#             print(species)
#             print(kmers)
#             print(counts)
            df1 = pd.DataFrame([counts], columns = kmers)
#             print(df1)
            merged = pd.merge(df, df1, how = 'outer').fillna(0)  #pd.concat([df1, df], ignote_index = True)
#             if np.shape(merged)[0]%2 != 0: # base test if we can merge the correct dfs correctly
#                 print("you screwed up!!!!!!!!!!!!!!!!!!")
            
#             print(merged)
#             updated_df = merged.head(int(np.shape(merged)[0]/2)).reset_index(drop = True)
#             updated_df1 = merged.tail(int(np.shape(merged)[0]/2)).reset_index(drop = True)
#             df = updated_df + updated_df1
            df = merged
#             print(df)
            combine_df = combine_df.append(df)
#             print(combine_df)
    combine_df = combine_df.fillna(0)
    final_row = combine_df.iloc[[1]]
#     print(final_row)
    for i in range(np.shape(combine_df)[0])[2:]:
#         print("finished processing this file, adding up and append this single row on to final_df")
        final_row = final_row + combine_df.iloc[[i]]
#         print(final_row)
    species = "{in1} {in2}".format(in1=description.split()[1], in2=description.split()[2])
    final_row['species'] = species
    final_df = final_df.append(final_row)  #remember to remove first row before data analysis!
#     print(final_df)

CPU times: user 5min 41s, sys: 3.33 s, total: 5min 44s
Wall time: 5min 44s


In [22]:
t_list = []
for t in df:
    t_list.append(t)
print(len(t_list))

257
